In [2]:
import sys, os, math, string, datetime as dt, time, re
import pandas as pd, numpy as np, matplotlib.pyplot as plt, sqlite3 as sql
import emosent, emoji, string
import yfinance as yf

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag, sentiment, RegexpParser, bigrams
from nltk.help import upenn_tagset

In [4]:
import csv, sys, re
from api_key import cryptor

In [5]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import bs4, pyautogui, requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
import random

In [6]:
# nltk.download("tagsets")

##### Parse Data From Discord Channel Scraper

In [7]:
tree = os.walk("../data/raw")

In [8]:
for folders, subfolders, files in tree:
    print(folders)
    print('--')
    print(subfolders)
    print('----')
    print(files)
    print("\n")

../data/raw
--
['BullTrades', 'Learn-n', 'Legacy Trading', 'LSV', 'Misc', 'OmegaTrades', 'Pennybois', 'Phantom Trades', 'Templar Trading', 'WayTrade']
----
['.gitkeep', 'crypt.db']


../data/raw\BullTrades
--
[]
----
['options-chat.db', 'stocks-chat.db']


../data/raw\Learn-n
--
[]
----
['trade-chat.db']


../data/raw\Legacy Trading
--
[]
----
['general.db', 'profits.db', 'text.db', 'trading-discussion.db']


../data/raw\LSV
--
[]
----
['options.db', 'trading.db']


../data/raw\Misc
--
[]
----
['wetlqd-ideas.db']


../data/raw\OmegaTrades
--
[]
----
['general.db', 'sentiment.db', 'tech.db', 'text.db', 'trade-ideas.db', 'winning.db']


../data/raw\Pennybois
--
[]
----
['stock-check.db']


../data/raw\Phantom Trades
--
[]
----
['lt-investing.db', 'trading-floor.db']


../data/raw\Templar Trading
--
[]
----
['day-trading.db', 'main-trading.db', 'option-trading.db']


../data/raw\WayTrade
--
[]
----
['general-trading.db']




In [9]:
def getalldata(folder):
    master_df = pd.DataFrame(columns=["id", "name", "content", "timestamp"])
    tree = os.walk(folder)
    for folders, subfolder, files in tree:
        if folders == "../data/raw":
            next
        else:
            
            for file in files: # get database file
                connection_str = folders.replace("\\", "/") +"/" + file #setup connection string
                connect = sql.connect(connection_str)
                mycur = connect.cursor()
                mycur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name") #get available tables
                available_tables = mycur.fetchall()
                server_name = folders.replace("../data/raw\\", "") #server name to append to df
                channel_name = file.replace(".db", "") # channel name to append to df
                for i, tbl in enumerate(available_tables):
                    df = pd.read_sql(f"SELECT * FROM {tbl[0]}", con=connect)
                    df["channel"] = channel_name
                    df["server"] = server_name
                    master_df = pd.concat([master_df, df], axis=0, ignore_index=True)
                connect.close()
                print(f"processed {channel_name}. Beep Bop Boop.....")
    return master_df

In [10]:
# Decompose Files to Singular DB w/ Columns to Represent the File Structure
total_df = getalldata("../data/raw")

processed options-chat. Beep Bop Boop.....
processed stocks-chat. Beep Bop Boop.....
processed trade-chat. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed profits. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trading-discussion. Beep Bop Boop.....
processed options. Beep Bop Boop.....
processed trading. Beep Bop Boop.....
processed wetlqd-ideas. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed sentiment. Beep Bop Boop.....
processed tech. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trade-ideas. Beep Bop Boop.....
processed winning. Beep Bop Boop.....
processed stock-check. Beep Bop Boop.....
processed lt-investing. Beep Bop Boop.....
processed trading-floor. Beep Bop Boop.....
processed day-trading. Beep Bop Boop.....
processed main-trading. Beep Bop Boop.....
processed option-trading. Beep Bop Boop.....
processed general-trading. Beep Bop Boop.....


In [11]:
# Comments
# Servers Scraped 
total_df.server.unique()

array(['BullTrades', 'Learn-n', 'Legacy Trading', 'LSV', 'Misc',
       'OmegaTrades', 'Pennybois', 'Phantom Trades', 'Templar Trading',
       'WayTrade'], dtype=object)

In [12]:
# Comments
# Channels Scraped
total_df.channel.unique()

array(['options-chat', 'stocks-chat', 'trade-chat', 'general', 'profits',
       'text', 'trading-discussion', 'options', 'trading', 'wetlqd-ideas',
       'sentiment', 'tech', 'trade-ideas', 'winning', 'stock-check',
       'lt-investing', 'trading-floor', 'day-trading', 'main-trading',
       'option-trading', 'general-trading'], dtype=object)

In [13]:
# Comments
total_df.server = total_df.server.apply(lambda x: x if x != "Misc" else "misc")

In [14]:
# Comments
total_df = total_df.sort_values(["server", "channel"])

In [17]:
# Comments
total_df.drop_duplicates(subset=["id", "content"])[lambda x: (x.server == "misc")].channel.value_counts()

wetlqd-ideas    4905
Name: channel, dtype: int64

##### Dealing with Duplicate Dbs in /Misc

In [18]:
# Comments
# stock-chat.db = Legacy Trading/profits.db = text3.db
total_df[lambda x: x.channel == "stock-chat"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

,id,name,content,timestamp,channel,server


In [14]:
# Comments
(total_df[lambda x: (x.channel == "profits")&(x.server == "Legacy Trading")]).drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

content  \
41234                                          Damn nice   
41229                              What’d you pay for it   
41225                                      FB looks good   
41221                                    2.55 as it said   
41217                             510 for both contracts   
...                                                  ...   
38934  AMD is wedging and this supply of confluent wi...   
38930  /NQ at a fresh 4h demands hopefully we can see...   
38927  /ES and /NQ are flying. No idea why equities a...   
38928  I’ll be posting some juicy charting and more p...   
38929                                               Same   

                              timestamp  channel          server  
41234  2020-10-14T02:30:06.383000+00:00  profits  Legacy Trading  
41229  2020-10-14T02:30:29.588000+00:00  profits  Legacy Trading  
41225  2020-10-14T02:30:32.627000+00:00  profits  Legacy Trading  
41221  2020-10-14T02:30:42.804000+00:00  profits  Legacy Trading  
41217  2020-10-14T02:30:46.699000+00:00  profits  Legacy Trading  
...                                 ...      ...             ...  
38934  2020-12-08T15:18:52.685000+00:00  profits  Legacy Trading  
38930  2020-12-08T15:21:21.489000+00:00  profits  Legacy Trading  
38927  2020-12-08T16:03:05.514000+00:00  profits  Legacy Trading  
38928  2020-12-08T16:37:44.868000+00:00  profits  Legacy Trading  
38929  2020-12-08T16:52:40.462000+00:00  profits  Legacy Trading  

[486 rows x 6 columns]

In [15]:
# Comments
# verdict: text3.db = stock-chat = Legacy Trading/profits
total_df[lambda x: x.channel == "text3"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

content  \
66444                                          Damn nice   
66439                              What’d you pay for it   
66435                                      FB looks good   
66431                                    2.55 as it said   
66427                             510 for both contracts   
...                                                  ...   
64144  AMD is wedging and this supply of confluent wi...   
64140  /NQ at a fresh 4h demands hopefully we can see...   
64137  /ES and /NQ are flying. No idea why equities a...   
64138  I’ll be posting some juicy charting and more p...   
64139                                               Same   

                              timestamp channel server  
66444  2020-10-14T02:30:06.383000+00:00   text3   misc  
66439  2020-10-14T02:30:29.588000+00:00   text3   misc  
66435  2020-10-14T02:30:32.627000+00:00   text3   misc  
66431  2020-10-14T02:30:42.804000+00:00   text3   misc  
66427  2020-10-14T02:30:46.699000+00:00   text3   misc  
...                                 ...     ...    ...  
64144  2020-12-08T15:18:52.685000+00:00   text3   misc  
64140  2020-12-08T15:21:21.489000+00:00   text3   misc  
64137  2020-12-08T16:03:05.514000+00:00   text3   misc  
64138  2020-12-08T16:37:44.868000+00:00   text3   misc  
64139  2020-12-08T16:52:40.462000+00:00   text3   misc  

[486 rows x 6 columns]

<h3 style="color:red">verdict: delete text3.db and stock-chat.db from /Misc/</h3>

In [16]:
# Comments
# OmegaTrades/potentialdupe = misc/trade-ideas.db = misc/text5.db 
total_df[lambda x: x.channel == "potentialdupe"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

content  \
159074  https://twitter.com/realdonaldtrump/status/131...   
159068  https://www.bloomberg.com/amp/news/articles/20...   
159069   So why are stocks mooning with this election bs?   
159070  > So why are stocks mooning with this election...   
159063                  Biden’s lead is good for big tech   
...                                                   ...   
158558                           holy shit read that yall   
158554       Crazy, why do you think whoever it was sold?   
158551  <@!548320800202293258> bc they think the marke...   
158552  nah but idk, they always sell before we know,t...   
158553                                            Ah true   

                               timestamp        channel       server  
159074  2020-10-06T18:54:07.769000+00:00  potentialdupe  OmegaTrades  
159068  2020-11-05T01:54:47.610000+00:00  potentialdupe  OmegaTrades  
159069  2020-11-06T03:02:40.575000+00:00  potentialdupe  OmegaTrades  
159070  2020-11-06T04:03:00.627000+00:00  potentialdupe  OmegaTrades  
159063  2020-11-06T04:03:46.984000+00:00  potentialdupe  OmegaTrades  
...                                  ...            ...          ...  
158558  2020-12-04T21:46:05.806000+00:00  potentialdupe  OmegaTrades  
158554  2020-12-05T03:22:51.893000+00:00  potentialdupe  OmegaTrades  
158551  2020-12-05T03:23:12.691000+00:00  potentialdupe  OmegaTrades  
158552  2020-12-05T03:24:23.736000+00:00  potentialdupe  OmegaTrades  
158553  2020-12-05T03:24:26.764000+00:00  potentialdupe  OmegaTrades  

[110 rows x 6 columns]

In [17]:
# Comments
# the trade ideas databate contains more unique comments
total_df[lambda x: x.channel == "trade-ideas"].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

content  \
110841  https://twitter.com/realdonaldtrump/status/131...   
111887                                         GME halted   
111878  https://www.google.com/amp/s/www.bloomberg.com...   
111873  Waymo is an autonomous driving tech company un...   
111868  https://t.co/u4ZbtW9JX5?amp=1 \n\n\nThe Compan...   
...                                                   ...   
110325                           holy shit read that yall   
110321       Crazy, why do you think whoever it was sold?   
110318  <@!548320800202293258> bc they think the marke...   
110319  nah but idk, they always sell before we know,t...   
110320                                            Ah true   

                               timestamp      channel server  
110841  2020-10-06T18:54:07.769000+00:00  trade-ideas   misc  
111887  2020-10-08T18:07:49.653000+00:00  trade-ideas   misc  
111878  2020-10-10T20:10:08.122000+00:00  trade-ideas   misc  
111873  2020-10-10T20:11:13.856000+00:00  trade-ideas   misc  
111868  2020-10-14T19:48:17.976000+00:00  trade-ideas   misc  
...                                  ...          ...    ...  
110325  2020-12-04T21:46:05.806000+00:00  trade-ideas   misc  
110321  2020-12-05T03:22:51.893000+00:00  trade-ideas   misc  
110318  2020-12-05T03:23:12.691000+00:00  trade-ideas   misc  
110319  2020-12-05T03:24:23.736000+00:00  trade-ideas   misc  
110320  2020-12-05T03:24:26.764000+00:00  trade-ideas   misc  

[222 rows x 6 columns]

In [18]:
# Comments
# trade-ideas = text5.db 
total_df[lambda x: (x.channel == "text5") & (x.server == "misc")].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

content  \
71008  https://twitter.com/realdonaldtrump/status/131...   
72054                                         GME halted   
72045  https://www.google.com/amp/s/www.bloomberg.com...   
72040  Waymo is an autonomous driving tech company un...   
72035  https://t.co/u4ZbtW9JX5?amp=1 \n\n\nThe Compan...   
...                                                  ...   
70492                           holy shit read that yall   
70488       Crazy, why do you think whoever it was sold?   
70485  <@!548320800202293258> bc they think the marke...   
70486  nah but idk, they always sell before we know,t...   
70487                                            Ah true   

                              timestamp channel server  
71008  2020-10-06T18:54:07.769000+00:00   text5   misc  
72054  2020-10-08T18:07:49.653000+00:00   text5   misc  
72045  2020-10-10T20:10:08.122000+00:00   text5   misc  
72040  2020-10-10T20:11:13.856000+00:00   text5   misc  
72035  2020-10-14T19:48:17.976000+00:00   text5   misc  
...                                 ...     ...    ...  
70492  2020-12-04T21:46:05.806000+00:00   text5   misc  
70488  2020-12-05T03:22:51.893000+00:00   text5   misc  
70485  2020-12-05T03:23:12.691000+00:00   text5   misc  
70486  2020-12-05T03:24:23.736000+00:00   text5   misc  
70487  2020-12-05T03:24:26.764000+00:00   text5   misc  

[222 rows x 6 columns]

In [19]:
# Comments
# OmegaTrades/potentialdupe = partial duplicate of misc/trade-ideas.db = misc/text5.db that are partial duplicates of text.db
total_df[lambda x: (x.channel == "text") & (x.server == "misc")].drop_duplicates(subset=["id", "content"]).sort_values("timestamp")

content  \
60604  https://twitter.com/realdonaldtrump/status/131...   
60598  https://www.bloomberg.com/amp/news/articles/20...   
60599   So why are stocks mooning with this election bs?   
60600  > So why are stocks mooning with this election...   
60593                  Biden’s lead is good for big tech   
...                                                  ...   
62112                                               FSLY   
62105                                               HYLN   
62102  Cheap leap that will guarantee a profit. IVR r...   
62100  Very bullish. Broke out and held it's place ab...   
62101                                    S&P 500 futures   

                              timestamp channel server  
60604  2020-10-06T18:54:07.769000+00:00    text   misc  
60598  2020-11-05T01:54:47.610000+00:00    text   misc  
60599  2020-11-06T03:02:40.575000+00:00    text   misc  
60600  2020-11-06T04:03:00.627000+00:00    text   misc  
60593  2020-11-06T04:03:46.984000+00:00    text   misc  
...                                 ...     ...    ...  
62112  2020-12-08T15:35:58.628000+00:00    text   misc  
62105  2020-12-08T15:40:38.188000+00:00    text   misc  
62102  2020-12-08T16:10:49.635000+00:00    text   misc  
62100  2020-12-08T16:24:27.489000+00:00    text   misc  
62101  2020-12-08T16:24:30.881000+00:00    text   misc  

[439 rows x 6 columns]

<h3 style="color:red">verdict: delete OmegaTrades/potentialdupe.db, Misc/text5.db and ./trade-ideas.db + rename: text.db to trade-ideas and mv to /OmegaTrades</h3>

In [20]:
# Comments
# Omega Trades/winning = misc/text2.db
total_df[lambda x: (x.channel == "text2") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

content  \
63613                                  another good call   
63614              PLTR this morning and a DKNG daytrade   
63617                                Quick trade on DDOG   
63620  200 scalping DOCU\nLots of other trades that u...   
63623                                      Snap lottos 🙂   
...                                                  ...   
64107  250 on AMD quick scalp play this morning just ...   
64117            <@!673279340326420549> Thank you sir ^^   
64122  $60.00 profit. \nBut bad risk to reward ratio\...   
64124             Entered too early then exited to early   
64134                       I’ve been on fire this week🔥   

                              timestamp channel server  
63613  2020-12-08T15:18:53.907000+00:00   text2   misc  
63614  2020-12-08T16:35:06.936000+00:00   text2   misc  
63617  2020-12-04T16:32:40.061000+00:00   text2   misc  
63620  2020-12-04T16:11:53.140000+00:00   text2   misc  
63623  2020-12-03T16:58:01.890000+00:00   text2   misc  
...                                 ...     ...    ...  
64107  2020-10-26T13:39:52.625000+00:00   text2   misc  
64117  2020-10-21T22:38:42.152000+00:00   text2   misc  
64122  2020-10-21T15:23:17.722000+00:00   text2   misc  
64124  2020-10-21T14:58:06.478000+00:00   text2   misc  
64134  2020-10-16T13:38:35.426000+00:00   text2   misc  

[105 rows x 6 columns]

In [21]:
# Comments
# Winning has a few more comments
(total_df[lambda x: (x.channel == "winning")&(x.server == "OmegaTrades")]).drop_duplicates(subset=["id", "content"])

content  \
167637                                  another good call   
167638              PLTR this morning and a DKNG daytrade   
167641                                Quick trade on DDOG   
167644  200 scalping DOCU\nLots of other trades that u...   
167647                                      Snap lottos 🙂   
...                                                   ...   
168181             Let’s go hippo base hits not home runs   
168183  It’s not much but I’m starting to get the ball...   
168189                                      Apple 116-123   
168192                           512.90 x 80\n513.19 x 79   
168199                          Quick ZM scalp cashed out   

                               timestamp  channel       server  
167637  2020-12-08T15:18:53.907000+00:00  winning  OmegaTrades  
167638  2020-12-08T16:35:06.936000+00:00  winning  OmegaTrades  
167641  2020-12-04T16:32:40.061000+00:00  winning  OmegaTrades  
167644  2020-12-04T16:11:53.140000+00:00  winning  OmegaTrades  
167647  2020-12-03T16:58:01.890000+00:00  winning  OmegaTrades  
...                                  ...      ...          ...  
168181  2020-10-14T14:49:11.936000+00:00  winning  OmegaTrades  
168183  2020-10-14T14:48:51.657000+00:00  winning  OmegaTrades  
168189  2020-10-13T15:48:46.812000+00:00  winning  OmegaTrades  
168192  2020-10-13T15:47:09.237000+00:00  winning  OmegaTrades  
168199  2020-10-13T14:25:27.390000+00:00  winning  OmegaTrades  

[114 rows x 6 columns]

<h3 style="color:red">verdict: delete Misc/text2.db</h3>

In [22]:
# Comments
# text.db = text4.db
total_df[lambda x: (x.channel == "text") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

content  \
59577  <@!548320800202293258> bc they think the marke...   
59578  nah but idk, they always sell before we know,t...   
59579                                            Ah true   
59580       Crazy, why do you think whoever it was sold?   
59584                           holy shit read that yall   
...                                                  ...   
63588  AAPL: Wait for a break to either the upside or...   
63593             MSFT: Trading Sideways, wait for break   
63598  GG <@!374316215797284877>, you just advanced t...   
63603  Costco: Ascending triangle, watch for a break ...   
63608                                    Ya, gimme a sec   

                              timestamp channel server  
59577  2020-12-05T03:23:12.691000+00:00    text   misc  
59578  2020-12-05T03:24:23.736000+00:00    text   misc  
59579  2020-12-05T03:24:26.764000+00:00    text   misc  
59580  2020-12-05T03:22:51.893000+00:00    text   misc  
59584  2020-12-04T21:46:05.806000+00:00    text   misc  
...                                 ...     ...    ...  
63588  2020-11-21T22:29:21.835000+00:00    text   misc  
63593  2020-11-21T22:29:10.607000+00:00    text   misc  
63598  2020-11-21T22:29:00.484000+00:00    text   misc  
63603  2020-11-21T22:29:00.368000+00:00    text   misc  
63608  2020-11-21T22:18:43.131000+00:00    text   misc  

[439 rows x 6 columns]

In [23]:
# Comments
total_df[lambda x: (x.channel == "text4") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

content  \
66449  <@!548320800202293258> bc they think the marke...   
66450  nah but idk, they always sell before we know,t...   
66451                                            Ah true   
66452       Crazy, why do you think whoever it was sold?   
66456                           holy shit read that yall   
...                                                  ...   
70460  AAPL: Wait for a break to either the upside or...   
70465             MSFT: Trading Sideways, wait for break   
70470  GG <@!374316215797284877>, you just advanced t...   
70475  Costco: Ascending triangle, watch for a break ...   
70480                                    Ya, gimme a sec   

                              timestamp channel server  
66449  2020-12-05T03:23:12.691000+00:00   text4   misc  
66450  2020-12-05T03:24:23.736000+00:00   text4   misc  
66451  2020-12-05T03:24:26.764000+00:00   text4   misc  
66452  2020-12-05T03:22:51.893000+00:00   text4   misc  
66456  2020-12-04T21:46:05.806000+00:00   text4   misc  
...                                 ...     ...    ...  
70460  2020-11-21T22:29:21.835000+00:00   text4   misc  
70465  2020-11-21T22:29:10.607000+00:00   text4   misc  
70470  2020-11-21T22:29:00.484000+00:00   text4   misc  
70475  2020-11-21T22:29:00.368000+00:00   text4   misc  
70480  2020-11-21T22:18:43.131000+00:00   text4   misc  

[439 rows x 6 columns]

<h3 style="color:red">remove Misc/text4.db</h3>

In [24]:
# Comments
# wetlqd-ideas = text6.db
total_df[lambda x: (x.channel == "text6") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

content  \
72058   Thanks <@464449226521313292> !  I opted for th...   
72059   Barely followed the market at all today (didn’...   
72060     <@131952052069728258> has reached level **13**!   
72063                  <@!464449226521313292> thanks boss   
72067   <@!399198468159045633> BRO AWSOME SO GLAD YOU ...   
...                                                   ...   
110279                                             Heroin   
110284                                          Big crack   
110289                                              Crack   
110294  @everyone today’s total. This doesn’t calculat...   
110307                                 Play account today   

                               timestamp channel server  
72058   2020-11-30T20:48:43.035000+00:00   text6   misc  
72059   2020-11-30T20:58:32.872000+00:00   text6   misc  
72060   2020-11-30T19:55:20.913000+00:00   text6   misc  
72063   2020-11-30T19:55:20.625000+00:00   text6   misc  
72067   2020-11-30T19:44:26.516000+00:00   text6   misc  
...                                  ...     ...    ...  
110279  2019-08-02T01:14:44.367000+00:00   text6   misc  
110284  2019-08-02T01:14:39.682000+00:00   text6   misc  
110289  2019-08-02T01:14:35.798000+00:00   text6   misc  
110294  2019-08-02T01:13:47.923000+00:00   text6   misc  
110307  2019-08-02T01:10:52.637000+00:00   text6   misc  

[4905 rows x 6 columns]

In [25]:
# Comments
total_df[lambda x: (x.channel == "wetlqd-ideas") & (x.server == "misc")].drop_duplicates(subset=["id", "content"])

content  \
111891  Thanks <@464449226521313292> !  I opted for th...   
111892  Barely followed the market at all today (didn’...   
111893    <@131952052069728258> has reached level **13**!   
111896                 <@!464449226521313292> thanks boss   
111900  <@!399198468159045633> BRO AWSOME SO GLAD YOU ...   
...                                                   ...   
150112                                             Heroin   
150117                                          Big crack   
150122                                              Crack   
150127  @everyone today’s total. This doesn’t calculat...   
150140                                 Play account today   

                               timestamp       channel server  
111891  2020-11-30T20:48:43.035000+00:00  wetlqd-ideas   misc  
111892  2020-11-30T20:58:32.872000+00:00  wetlqd-ideas   misc  
111893  2020-11-30T19:55:20.913000+00:00  wetlqd-ideas   misc  
111896  2020-11-30T19:55:20.625000+00:00  wetlqd-ideas   misc  
111900  2020-11-30T19:44:26.516000+00:00  wetlqd-ideas   misc  
...                                  ...           ...    ...  
150112  2019-08-02T01:14:44.367000+00:00  wetlqd-ideas   misc  
150117  2019-08-02T01:14:39.682000+00:00  wetlqd-ideas   misc  
150122  2019-08-02T01:14:35.798000+00:00  wetlqd-ideas   misc  
150127  2019-08-02T01:13:47.923000+00:00  wetlqd-ideas   misc  
150140  2019-08-02T01:10:52.637000+00:00  wetlqd-ideas   misc  

[4905 rows x 6 columns]

<h3 style="color:red">remove Misc/text6.db</h3>

<h3>/raw/data Cleanup</h3>
<ul>
<li>rm: text6, text5, text4, text3, text2, stock-chat, trade-ideas, OmegaTrades/potentialDup</li>
<li>rename: misc/text.db -> Omega Trades/trade-ideas.db</li>
</ul>

In [19]:
# Comments
# Retrieve All the Data
total_df_no_misc = getalldata("../data/raw")

processed options-chat. Beep Bop Boop.....
processed stocks-chat. Beep Bop Boop.....
processed trade-chat. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed profits. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trading-discussion. Beep Bop Boop.....
processed options. Beep Bop Boop.....
processed trading. Beep Bop Boop.....
processed wetlqd-ideas. Beep Bop Boop.....
processed general. Beep Bop Boop.....
processed sentiment. Beep Bop Boop.....
processed tech. Beep Bop Boop.....
processed text. Beep Bop Boop.....
processed trade-ideas. Beep Bop Boop.....
processed winning. Beep Bop Boop.....
processed stock-check. Beep Bop Boop.....
processed lt-investing. Beep Bop Boop.....
processed trading-floor. Beep Bop Boop.....
processed day-trading. Beep Bop Boop.....
processed main-trading. Beep Bop Boop.....
processed option-trading. Beep Bop Boop.....
processed general-trading. Beep Bop Boop.....


In [20]:
# Comments
# Remove Duplicates
total_df_no_misc = total_df_no_misc.drop_duplicates(["id", "content"])

In [21]:
# Comments
# Unique Channels / Topics
total_df_no_misc.channel.unique()

array(['options-chat', 'stocks-chat', 'trade-chat', 'general', 'profits',
       'text', 'trading-discussion', 'options', 'trading', 'wetlqd-ideas',
       'sentiment', 'tech', 'trade-ideas', 'winning', 'stock-check',
       'lt-investing', 'trading-floor', 'day-trading', 'main-trading',
       'option-trading', 'general-trading'], dtype=object)

In [22]:
# Comments
total_df_no_misc = total_df_no_misc.sort_values("timestamp").reset_index(drop=True)

In [23]:
# Comments
total_df_no_misc.shape

(31934, 6)

In [ ]:
# Comments
# Define Regular Expressions to find specialized grams / tokens.
# Urls
url_regex = r"https*\:.+|www\..+"
# Discord Emotes Token
discord_emote_regex = r"<:.+:\d+>"
# Names of Chat Specific Emotes
emote_name = r":\w+:"


In [ ]:
# Comments
# Function to Decompose Comments into New Columns
# Used to Prep Comments Best for Sentiment Analysis
def add_specialized_token_column(regex, comment_column, new_column_name):
    total_df.loc[:, new_column_name] = total_df.loc[:, comment_column].apply(lambda x: ",".join([r for r in re.findall(regex, x)]))
    return total_df

In [ ]:
# Comments
total_df_w_url = add_specialized_token_column(url_regex, "content", "links")
total_df_w_url.links.unique()

In [ ]:
# Comments
total_df_w_new_cols = add_specialized_token_column(discord_emote_regex, "content", "chat_emotes")
total_df_w_new_cols.chat_emotes.unique()

In [ ]:
# Comments
total_df_w_new_cols[lambda x: (x.chat_emotes!="")]

In [ ]:
# Comments
total_df_w_new_cols.loc[:, "chat_emotes"] = total_df_w_new_cols.chat_emotes.apply(lambda x: ",".join([e.replace(":", "") for e in re.findall(emote_name, x)]))
total_df_w_new_cols[lambda x: x.chat_emotes != ""]

<h3>Comment Cleaning</h3>
    <ul>
        <li>Starting Point: 31,934 Unique Comments</li>
        <li>In order to get these comments ready for NLP analysis, I must:</li>
        <ul>
        <li>Remove scripted comments (those automatically genreated by bots)</li>
        <li>Identify Single Words Making Up Comments That Are Key to This Analysis:</li>
            <ul>
            <li>Emojis and Emotes</li>
            <li>Specific Companies and Their Tickers</li>
            <li>User Mentions</li>
            <li>Links</li>
            </ul>
        </ul>
    </ul>

#### Find Mentions

In [344]:
# Comments
# A Reference to Specific Chat Users displayed in the chatroom as @user and in the scraped data as <@id>
# Catch All Mentions (User and Generic)
def catchMentions(comment):
    userIdCatcherT = r"(<@(!)?\d+>|@everyone)"
    mention = re.findall(userIdCatcherT, comment)
    if mention != []:
        return ",".join([re.sub(r"[<>!@]", "", x[i][0]) for i, x in enumerate([mention])])
    else:
        return ""

In [345]:
# Comments
total_df = total_df_no_misc.assign(mentions = lambda x: x.content.apply(catchMentions))

#### Find Bots

In [90]:
# Comments
# Find Automated Response from Non-human Chatters
# !abbv represent commands said in the channel in order to get a response from specific bot 
# total_df[lambda x: x.content.str.contains(r"!ta")].iloc[0, 2] # user commands to call bots ~ to a user mention 
def catchBot(c):
    botCom = re.compile("\shas reached level \*\*.\d+\*\*")
    bot2Com = re.compile("\<.*\>, you just advanced to")
    botreg = botCom.search(c)
    botreg1 = bot2Com.search(c.strip())
    if (botreg != None) or (botreg1 != None):
        return 1
    else:
        return 0

In [91]:
# Comments
# First Run to create record of users
total_df_no_misc = total_df_no_misc.assign(isBot = lambda x: x.content.apply(catchBot))

In [246]:
# Comments
# Get Bot by Discord Id
bots = total_df_no_misc[lambda x: x.isBot == 1].id.unique()

In [257]:
# Comments
# Combine IDs w/ Self Proclaimed Bots
bots = np.concatenate([bots, total_df_no_misc[lambda x: x.name.str.contains("Bot#")].id.unique()], axis=0)

In [258]:
# Comments
# Identify the Robots !
total_df_no_misc.loc[(total_df_no_misc.id.isin(bots)), "isBot"] = 1

#### Find Emojis

In [346]:
# Comments
# Catch the Emotes
# Good proxy for mood and feeling packaged in a single token / gram
def catchEmojis(comment):
    emoji_list = emoji.emoji_lis(comment)
    return ",".join([e["emoji"] for e in emoji_list])

In [347]:
# Comments
total_df = total_df.assign(emojis = lambda x: x.content.apply(catchEmojis))

In [348]:
# Comments
total_df[lambda x: x.emojis != ""].head()

,content,timestamp,channel,server,isBot,mentions,emojis
12,😃,2019-08-02T17:55:31.789000+00:00,wetlqd-ideas,Misc,0,,😃
41,Griffin McDonald’s is all doodoo 😂😂 <@57920539...,2019-08-08T17:02:22.143000+00:00,wetlqd-ideas,Misc,0,579205396166737921,"😂,😂"
80,🔥🔥😳😳,2019-08-16T15:19:59.020000+00:00,wetlqd-ideas,Misc,0,,"🔥,🔥,😳,😳"
87,😂,2019-08-16T16:37:53.126000+00:00,wetlqd-ideas,Misc,0,,😂
97,Good morning everyone 😃,2019-08-19T13:34:26.860000+00:00,wetlqd-ideas,Misc,0,,😃


#### Find Companies
* Find companies using name, ticker or associated words
    * This project goal is to match and process comment sentiment and match it with the particular companies mentioned.
    * Companies are signified by a variety of words including: their traded ticker abbreviation, their full company name and by the names of members of their C-Suite.
    * In order to accurately and systematically obtain these mentioned companies, I will have to create a simple list of companies based on single words (ticker).
    * Once I have an understanding of the most popular companies being talked about in the groups, I can create a more refined parsing function to identify words that most likely trace back to the company.
    * I want to maintain the structure of the comments throughout the analysis (maintain initial structure), so that my NLP analysis can grade sentiment based on the original comment's grammer and word usage

In [349]:
# Comments
# Discords link to stock starts with a $, but can only pull out 338 comments out of 30,000+. There should be more.
total_df[lambda x: x.content.str.contains(r"\$[A-Za-z]")]

content  \
246    Yea, literally all thanks to you guys. Just wa...   
610                                                $EXAS   
1372   quick scalp on $SCWX with Dell buying out the ...   
1921    $IRM got small position. Will hold it overnight.   
1930                                 $Dis, interesting 🤔   
...                                                  ...   
31855           $GRIL again today <@!101806058359496704>   
31856        $CANG up too <:peepoyay:735222523683405984>   
31858  $AESE for anyone that was playing it just went...   
31889  $RUN i had on a watchlist like last week but t...   
31916     $OXY broke the resistance line on Brad's chart   

                              timestamp         channel           server  \
246    2019-08-29T21:06:45.315000+00:00    wetlqd-ideas             Misc   
610    2019-10-02T19:12:03.224000+00:00    wetlqd-ideas             Misc   
1372   2019-12-23T18:48:28.342000+00:00    wetlqd-ideas             Misc   
1921   2020-02-25T20:58:08.310000+00:00    wetlqd-ideas             Misc   
1930   2020-02-26T17:04:51.441000+00:00    wetlqd-ideas             Misc   
...                                 ...             ...              ...   
31855  2020-12-08T20:13:38.815000+00:00    main-trading  Templar Trading   
31856  2020-12-08T20:13:38.984000+00:00    main-trading  Templar Trading   
31858  2020-12-08T20:13:53.474000+00:00    main-trading  Templar Trading   
31889  2020-12-08T20:20:47.630000+00:00  option-trading  Templar Trading   
31916  2020-12-08T20:26:51.453000+00:00         trading              LSV   

       isBot            mentions emojis  
246        0                             
610        0                             
1372       0                             
1921       0                             
1930       0                          🤔  
...      ...                 ...    ...  
31855      0  101806058359496704         
31856      0                             
31858      0                             
31889      0                             
31916      0                             

[338 rows x 9 columns]

In [19]:
# Comments
# The starting Regex for tickers
company_regex = re.compile(r"[A-Z]{2,4}")

In [380]:
# Comments
# identifying companies based on tickers. Better representative. 338 -> 6,438 
total_df[lambda x:(x.content.str.contains(company_regex))]

content  \
23     I took stamps .... and IAC both not looking go...   
26                Sold AMD aug23rd call for a fat profit   
47     MSFT profits im still not selling it tho just ...   
54                                       $139.50 and ATH   
56                                  $139.50 and ATH? Wym   
...                                                  ...   
31910                      @everyone EDIT is popping off   
31916     $OXY broke the resistance line on Brad's chart   
31919  if PIXY runs from being included in the Benzin...   
31930  <@428921853403463713> have you done a video in...   
31933                                             CRM ??   

                              timestamp       channel   server  isBot  \
23     2019-08-08T03:31:33.793000+00:00  wetlqd-ideas     Misc      0   
26     2019-08-08T13:33:00.968000+00:00  wetlqd-ideas     Misc      0   
47     2019-08-13T14:21:58.192000+00:00  wetlqd-ideas     Misc      0   
54     2019-08-13T19:01:16.234000+00:00  wetlqd-ideas     Misc      0   
56     2019-08-13T19:05:27.213000+00:00  wetlqd-ideas     Misc      0   
...                                 ...           ...      ...    ...   
31910  2020-12-08T20:25:14.542000+00:00       trading      LSV      0   
31916  2020-12-08T20:26:51.453000+00:00       trading      LSV      0   
31919  2020-12-08T20:27:59.342000+00:00    trade-chat  Learn-n      0   
31930  2020-12-08T20:39:17.246000+00:00       trading      LSV      0   
31933  2020-12-08T20:41:38.298000+00:00       options      LSV      0   

                 mentions emojis  
23                                
26                                
47                                
54                                
56                                
...                   ...    ...  
31910            everyone         
31916                             
31919                             
31930  428921853403463713         
31933                             

[6483 rows x 9 columns]

In [4]:
# next to do:
    # gather list of companies on the main American exchanges: NYSE and NASDAQ and their tickers (available via NASDAQ's website)
    # have dictionary parser that translates company name mentions to its associated ticker.
    # with the name and tickers, a more thorough pulling of the appropiate companies can take place. The pulling should take place on the str.lower value of the content column.
    # company mentions will be stored just as emojis and mentions are stored for now. (as comma-separated values column)

In [ ]:
# Comments
# Use the previously defined regular expressions to find the non-word tokens in a sentence
userIdCatcher = r"(<@(!)?\d+>|@everyone)"
discord_emote_regex = r"<:.+:\d+>"
url_regex = r"https*\:.+|www\..+"
punc_regex = r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_’]"
contract_regex = r"[\'\’\’]"
emoji_regex = emoji.get_emoji_regexp()
stop_words = set(stopwords.words("english"))

In [ ]:
# Comments
# standardize the tokens
def clean_comment(comment):
    comment = re.sub("|".join([x for x in [userIdCatcher, discord_emote_regex, url_regex]]), "", comment)
    comment = re.sub(contract_regex, "", comment)
    comment = re.sub(emoji_regex, "", comment)
    comment = re.sub(punc_regex, " ", comment)
    return comment.replace("\n", " ").lower()

In [37]:
# Comments
# understand basic linguistic structure of comments. Words used. To see how I should approach picking out companies and analyzing sentiment
def wordCounts(comment_array):
    stop_words = set(stopwords.words("english"))
    comment_dictionary = {} 
    for comment in comment_array:
        # remove punc
        punc_regex = r"[\’\'\'\"\"\“\”!\?@#$%&\(\)\*,-.\\\{\}+~\/:;<>\[\]^`|=_]"
        # lower here to standardize all words
        comment = comment.lower()
        comment = re.sub(punc_regex, "", comment)
        # remove numbers
        comment = re.sub(r"[0-9]", "", comment)
        # remove emojis
        emoji_list = emoji.emoji_lis(comment)
        emoji_list = [e["emoji"] for e in emoji_list]
        # tokenize 
        words = word_tokenize(comment)
        for w in words:
            if (w in comment_dictionary.keys()) & (w not in stop_words) & (w not in emoji_list):
                comment_dictionary[w] += 1
            elif (w not in stop_words) & (w not in emoji_list):
                comment_dictionary[w] = 1
    
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    # returns DF with token and count
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [37]:
# word counts sans stopwords
wordy_df = wordCounts(total_df.content.values)

In [37]:
# possible check for ticker mentions
wordy_df = wordy_df.assign(no_vow = lambda x: x.word.apply(lambda x: 1 if re.findall(r"[aeiou]+", x) != [] else 0))

In [37]:
# Comments
# Example Rough Cut
wordy_df[lambda x: x.no_vow <1].head(30)
total_df[lambda x: x.content.str.contains(r"\s*jks\s*")]

In [40]:
# Comments
# Show Tree Breakdown via part of speech tagging
sample_tokens = word_tokenize(total_df[lambda x: x.content.str.contains("amzn")].sample(1, random_state=6).content.values[0])
sample_pos_tags = pos_tag(sample_tokens)
pattern = 'NP: {<DT>?<JJ>*<NN>}'
comment_parse = RegexpParser(pattern)
tree = comment_parse.parse(sample_pos_tags)

In [40]:
# Comments Draw Tree
print(tree)
tree.draw()

In [ ]:
# Comments
# Create a rough Lexicon specific to our chatters
token_df = total_df_w_new_cols.copy(deep=True)
token_df = token_df[lambda x: x.isBot == 0]
token_df = token_df.assign(for_tokenizer = lambda x: x.content.apply(clean_comment)).reset_index(drop=True)
token_df.head()

In [879]:
# Comments
token_df.timestamp = pd.to_datetime(token_df.timestamp)

In [528]:
# Comments
token_df.groupby([pd.Grouper(key="timestamp", freq="M"), "server"]).count()

,,id,name,content,channel,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
timestamp,server,,,,,,,,,,
2019-08-31 00:00:00+00:00,Misc,264,264,264,264,264,264,264,264,264,264
2019-09-30 00:00:00+00:00,Misc,318,318,318,318,318,318,318,318,318,318
2019-10-31 00:00:00+00:00,Misc,303,303,303,303,303,303,303,303,303,303
2019-11-30 00:00:00+00:00,Misc,307,307,307,307,307,307,307,307,307,307
2019-12-31 00:00:00+00:00,Misc,226,226,226,226,226,226,226,226,226,226
2020-01-31 00:00:00+00:00,Misc,236,236,236,236,236,236,236,236,236,236
2020-02-29 00:00:00+00:00,Misc,321,321,321,321,321,321,321,321,321,321
2020-03-31 00:00:00+00:00,Misc,458,458,458,458,458,458,458,458,458,458
2020-04-30 00:00:00+00:00,Misc,366,366,366,366,366,366,366,366,366,366


In [880]:
# Comments
token_df.groupby([pd.Grouper(key="timestamp", freq="M"), "server"]).count()

,,id,name,content,channel,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
timestamp,server,,,,,,,,,,
2019-08-31 00:00:00+00:00,Misc,264,264,264,264,264,264,264,264,264,264
2019-09-30 00:00:00+00:00,Misc,318,318,318,318,318,318,318,318,318,318
2019-10-31 00:00:00+00:00,Misc,303,303,303,303,303,303,303,303,303,303
2019-11-30 00:00:00+00:00,Misc,307,307,307,307,307,307,307,307,307,307
2019-12-31 00:00:00+00:00,Misc,226,226,226,226,226,226,226,226,226,226
2020-01-31 00:00:00+00:00,Misc,236,236,236,236,236,236,236,236,236,236
2020-02-29 00:00:00+00:00,Misc,321,321,321,321,321,321,321,321,321,321
2020-03-31 00:00:00+00:00,Misc,458,458,458,458,458,458,458,458,458,458
2020-04-30 00:00:00+00:00,Misc,366,366,366,366,366,366,366,366,366,366


In [538]:
# Comments
# The people love elon
token_df[lambda x: x.for_tokenizer.str.contains("elon")].loc[3651, "content"]

'Ya NeuroLink is what I’m investing in too, I mean who wouldn’t as we near the technological singularity. Elon is just one of many people who will change the world❤️'

In [574]:
# Comments
token_df.loc[7733, "content"]

'https://www.globenewswire.com/news-release/2020/09/21/2096779/0/en/Nikola-Corporation-Investors-Lawsuit-filed-to-recover-losses-suffered-as-a-result-of-alleged-fraud.html'

In [626]:
# Comments
token_df.loc[7790, "content"]

'market should rally before elections'

In [627]:
sample_tokens = word_tokenize(token_df.loc[7790, "content"])
sample_pos_tags = pos_tag(sample_tokens)

In [628]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

In [629]:
comment_parse = RegexpParser(pattern)
tree = comment_parse.parse(sample_pos_tags)

In [630]:
print(tree)

(S (NP market/NN) should/MD rally/VB before/IN elections/NNS)


In [657]:
tree.draw()

In [ ]:
# Comments
lex_df = wordCounts(token_df.for_tokenizer.values)

In [856]:
# Comments
lex_df.shape

(13868, 2)

In [857]:
# Comments
# sample of key company tokens
lex_df[lambda x: x.word == "amzn"] # stock ticker

,word,count
498,amzn,60


In [861]:
# Comments
# sample of key company tokens
lex_df[lambda x: x.word == "tsla"] # stock ticker

,word,count
100,tsla,254


In [ ]:
# The most important tokens left to be selected are urls, discord specific emotes (<:[A-Za-z]+:\d+>) and finally companies mentioned
# Being able to best ascertain the companies mentioned  by specific comments (company name and company ticker) will allow me to begin to collect the associated industry sentiment,
# which will be the daily finanical data of the most popular companies, the Financial News during this period and its sentiment and Institutional Analyst Ratings from Investment Banks and Research Facilities
# These three separate data sources will provide sentiment scores representing retail investor sentiment (from the discord private chats), finicial news sector sentiment (from the scraping of the Financial News)
# and the institutional players Sentiment for the analyzed Companies.

# The three sentiments of the separate sectors will give us the general mood of these sectors in time to match with the financial data for a company during this same time.
# Having these separate data souces broken down by type will allow us to tell the story of a company by the moods of the separate cohorts. 

In [ ]:
# Finding Companies 
# We want to get a list of all tokens and their frequencies as the starting point to company mentions.
# to do so, we need to copy the content column, lower its words, strip out the specialized tokens in a row's columns, remove all puncutations and remove stop words.
# From here, we should get a good starting lexicon for the chatrooms and analyze it to see which tokens refer back to a specific company

In [865]:
# Comments Create Bigrams
def pairCounts(comment_array):
    comment_dictionary = {} 
    for comment in comment_array:
        words = word_tokenize(comment)
        pairs = [" ".join(pair) for pair in bigrams(words)]
        for pair in pairs:
            couple = pair.split(" ")
            if (pair in comment_dictionary.keys()) & (couple[0] not in stop_words) & (couple[1] not in stop_words):
                comment_dictionary[pair] += 1
            elif (pair not in stop_words) & (couple[0] not in stop_words) & (couple[1] not in stop_words):
                comment_dictionary[pair] = 1
    comments_words = dict(sorted(comment_dictionary.items(), key=lambda x: x[1], reverse=True))
    return pd.DataFrame.from_dict(comments_words, orient="index", columns=["count"]).reset_index().rename(columns={"index": "word"})

In [866]:
# Comments
word_pairs = pairCounts(token_df.for_tokenizer.values)

In [867]:
# Comments
word_pairs.head(25)

,word,count
0,gon na,382
1,long term,208
2,next week,147
3,looks like,115
4,got ta,109
5,day trade,107
6,dont think,83
7,dont know,74
8,wan na,71
9,last week,70


In [446]:
# Comments
ticker_array = tickers_and_names.ticker.values

In [535]:
# Comments
tickers_and_names[lambda x: x.ticker == "nio"]

,Symbol,Name,single_name,ticker
215,NIO,NIO Inc. American depositary shares each repr...,nio american each representing one ordinar...,nio


In [687]:
# Comments
 ######## del
company_regex = re.compile(r"[A-Z]{2,5}")

In [884]:
# Comments
# with Ticker mentions
token_df[lambda x: x.content.str.contains(company_regex)]

content  \
23     I took stamps .... and IAC both not looking go...   
26                Sold AMD aug23rd call for a fat profit   
47     MSFT profits im still not selling it tho just ...   
54                                       $139.50 and ATH   
56                                  $139.50 and ATH? Wym   
...                                                  ...   
31617                      @everyone EDIT is popping off   
31623     $OXY broke the resistance line on Brad's chart   
31626  if PIXY runs from being included in the Benzin...   
31637  <@428921853403463713> have you done a video in...   
31640                                             CRM ??   

                             timestamp       channel   server  isBot  \
23    2019-08-08 03:31:33.793000+00:00  wetlqd-ideas     Misc      0   
26    2019-08-08 13:33:00.968000+00:00  wetlqd-ideas     Misc      0   
47    2019-08-13 14:21:58.192000+00:00  wetlqd-ideas     Misc      0   
54    2019-08-13 19:01:16.234000+00:00  wetlqd-ideas     Misc      0   
56    2019-08-13 19:05:27.213000+00:00  wetlqd-ideas     Misc      0   
...                                ...           ...      ...    ...   
31617 2020-12-08 20:25:14.542000+00:00       trading      LSV      0   
31623 2020-12-08 20:26:51.453000+00:00       trading      LSV      0   
31626 2020-12-08 20:27:59.342000+00:00    trade-chat  Learn-n      0   
31637 2020-12-08 20:39:17.246000+00:00       trading      LSV      0   
31640 2020-12-08 20:41:38.298000+00:00       options      LSV      0   

                 mentions emojis links chat_emotes  \
23                                                   
26                                                   
47                                                   
54                                                   
56                                                   
...                   ...    ...   ...         ...   
31617            everyone                            
31623                                                
31626                                                
31637  428921853403463713                            
31640                                                

                                           for_tokenizer  
23     i took stamps      and iac both not looking go...  
26                sold amd aug23rd call for a fat profit  
47     msft profits im still not selling it tho just ...  
54                                        139 50 and ath  
56                                   139 50 and ath  wym  
...                                                  ...  
31617                                edit is popping off  
31623       oxy broke the resistance line on brads chart  
31626  if pixy runs from being included in the benzin...  
31637       have you done a video in gogo by any chance   
31640                                             crm     

[6349 rows x 12 columns]

In [885]:
# Comments
# look through comments example
token_df[lambda x: x.for_tokenizer.str.contains(r" h ")]

,content,timestamp,channel,server,isBot,mentions,emojis,links,chat_emotes,for_tokenizer
7588,This is what I saw. Cnblast saw the opposite i...,2020-09-21 03:38:28.525000+00:00,options-chat,BullTrades,0,,,,,this is what i saw cnblast saw the opposite i...
8388,C&H on NFLX 4H?,2020-09-26 03:10:46.953000+00:00,options-chat,BullTrades,0,,,,,c h on nflx 4h
10225,were we bamboozled by that c&h breakout? <@!49...,2020-10-05 01:29:34.167000+00:00,general-trading,WayTrade,0,494525003484889088,,,,were we bamboozled by that c h breakout
10640,Hey I’m not gonna lie the inverse h&l actually...,2020-10-06 22:54:18.693000+00:00,trading-discussion,Legacy Trading,0,,🤣,,,hey im not gonna lie the inverse h l actually ...
11094,<@!693425461774123068> Do you see AREC as a lo...,2020-10-08 04:57:59.234000+00:00,day-trading,Templar Trading,0,693425461774123068,,,,do you see arec as a longer hold based on a ...
11289,"letters = [""A"", ""B"", ""C"", ""D"", ""E"", ""F"", ""G"", ...",2020-10-09 03:40:13.839000+00:00,main-trading,Templar Trading,0,,,,,letters a b c d e f g ...
12329,Inverse H&S played out today,2020-10-13 03:23:06.356000+00:00,options-chat,BullTrades,0,,,,,inverse h s played out today
15082,thats h ow you get blocked,2020-10-22 03:56:46.384000+00:00,trading-discussion,Legacy Trading,0,,,,,thats h ow you get blocked
15619,Looks like potential C&H on JMIA so should run...,2020-10-23 21:49:30.747000+00:00,stocks-chat,BullTrades,0,,,,,looks like potential c h on jmia so should run...
24584,SPLK for a swing. Daily chart might have an in...,2020-11-21 01:33:11.696000+00:00,profits,Legacy Trading,0,,,,,splk for a swing daily chart might have an in...


In [886]:
# Comments
# look through comments 
token_df[lambda x: x.for_tokenizer.str.contains(r"disney")].iloc[-3, 2]

'FRX-U: SPAC lead by SHAQ:\n\nThe basketball superstar Shaquille O\'Neal, Martin Luther King III, and three former Disney executives are joining forces to back a new blank-check company.\nForest Road Acquisition Corp. aims to raise $250 million for deals in the tech and media space.\nPotential targets include intellectual property, "audience aggregation platforms," and "companies in need of capital,"'

In [887]:
# Comments
# Token DF
token_df = token_df.assign(named_companies = lambda x: x.content.apply(lambda x: ",".join([x for x in re.findall(company_regex, x)])))

In [888]:
# Comments
# Find the companies
c_dict = {}
for val in token_df.named_companies.values:
    if "," in val:
        for v in val.split(","):
            if v in c_dict.keys():
                c_dict[v] += 1
            else:
                c_dict[v] = 1
    else:
        if val in c_dict.keys():
            c_dict[val] += 1
        else:
            c_dict[val] = 1

In [889]:
# Comments
# Find the Companies
company_starter_df = pd.DataFrame.from_dict(c_dict, orient="index", columns=["counts"]).reset_index().rename(columns={"index": "company"}).sort_values("counts", ascending=False).reset_index(drop=True)

In [890]:
# Comments
# Top 25 Companies 
company_starter_df.head(25)

,company,counts
0,,25292
1,SPY,166
2,TSLA,165
3,NIO,151
4,AMD,133
5,AAPL,114
6,ER,90
7,AH,86
8,DKNG,75
9,PLTR,74


In [891]:
# Comments
# Regex Identifies ~2330 ticker-like tokens
# needs to be cleaned
company_starter_df.tail(40)

,company,counts
2291,HYG,1
2292,FUCKE,1
2293,ROCKE,1
2294,XLU,1
2295,EXC,1
2296,DSGT,1
2297,XLV,1
2298,FRTA,1
2299,QE,1
2300,SENAT,1


In [892]:
# Comments
# Examples of abbreviations that do not point to an actual company (even if the ticker may be listed)
# rows content can see the comments are talking about earnings reports
token_df[lambda x: x.named_companies.str.contains("ER")]

content  \
283                              Crwd over 120+ after ER   
578     ER play. Sold too early but still a big win #MTN   
793                                          SLIGHT WERK   
802         Post ER trade this morning for quick profits   
820                                             Tesla ER   
...                                                  ...   
29578      I will play ER's when my research confirms it   
29749          I’ve been VERY green this week, hb yall ?   
30232                              TANK VERY.CN INSTEAD!   
30469  i stay away from Forex for now - and i would d...   
31086                                        WHEREEEEEEE   

                             timestamp       channel           server  isBot  \
283   2019-09-04 16:35:46.747000+00:00  wetlqd-ideas             Misc      0   
578   2019-09-28 00:42:56.186000+00:00  wetlqd-ideas             Misc      0   
793   2019-10-21 19:32:02.200000+00:00  wetlqd-ideas             Misc      0   
802   2019-10-22 15:16:30.915000+00:00  wetlqd-ideas             Misc      0   
820   2019-10-24 13:40:17.450000+00:00  wetlqd-ideas             Misc      0   
...                                ...           ...              ...    ...   
29578 2020-12-04 04:44:08.376000+00:00       options              LSV      0   
29749 2020-12-04 19:50:40.190000+00:00       general      OmegaTrades      0   
30232 2020-12-05 23:30:35.028000+00:00  main-trading  Templar Trading      0   
30469 2020-12-06 20:32:12.681000+00:00    trade-chat          Learn-n      0   
31086 2020-12-08 04:06:00.473000+00:00  main-trading  Templar Trading      0   

      mentions emojis links chat_emotes  \
283                                       
578                                       
793                                       
802                                       
820                                       
...        ...    ...   ...         ...   
29578                                     
29749                                     
30232                                     
30469                                     
31086                                     

                                           for_tokenizer  \
283                              crwd over 120  after er   
578     er play  sold too early but still a big win  mtn   
793                                          slight werk   
802         post er trade this morning for quick profits   
820                                             tesla er   
...                                                  ...   
29578       i will play ers when my research confirms it   
29749           ive been very green this week  hb yall     
30232                              tank very cn instead    
30469  i stay away from forex for now   and i would d...   
31086                                        whereeeeeee   

             named_companies  
283                       ER  
578                   ER,MTN  
793               SLIGH,WERK  
802                       ER  
820                       ER  
...                      ...  
29578                     ER  
29749                   VERY  
30232  TANK,VERY,CN,INSTE,AD  
30469                   VERY  
31086            WHERE,EEEEE  

[154 rows x 13 columns]

In [807]:
# Comments
# find the uppercase freakouts
token_df.assign(isUpper = lambda x:x.content.apply(str.isupper))

content  \
23     I took stamps .... and IAC both not looking go...   
26                Sold AMD aug23rd call for a fat profit   
47     MSFT profits im still not selling it tho just ...   
54                                       $139.50 and ATH   
56                                  $139.50 and ATH? Wym   
...                                                  ...   
31617                      @everyone EDIT is popping off   
31623     $OXY broke the resistance line on Brad's chart   
31626  if PIXY runs from being included in the Benzin...   
31637  <@428921853403463713> have you done a video in...   
31640                                             CRM ??   

                             timestamp       channel   server  isBot  \
23    2019-08-08 03:31:33.793000+00:00  wetlqd-ideas     Misc      0   
26    2019-08-08 13:33:00.968000+00:00  wetlqd-ideas     Misc      0   
47    2019-08-13 14:21:58.192000+00:00  wetlqd-ideas     Misc      0   
54    2019-08-13 19:01:16.234000+00:00  wetlqd-ideas     Misc      0   
56    2019-08-13 19:05:27.213000+00:00  wetlqd-ideas     Misc      0   
...                                ...           ...      ...    ...   
31617 2020-12-08 20:25:14.542000+00:00       trading      LSV      0   
31623 2020-12-08 20:26:51.453000+00:00       trading      LSV      0   
31626 2020-12-08 20:27:59.342000+00:00    trade-chat  Learn-n      0   
31637 2020-12-08 20:39:17.246000+00:00       trading      LSV      0   
31640 2020-12-08 20:41:38.298000+00:00       options      LSV      0   

                 mentions emojis links chat_emotes  \
23                                                   
26                                                   
47                                                   
54                                                   
56                                                   
...                   ...    ...   ...         ...   
31617            everyone                            
31623                                                
31626                                                
31637  428921853403463713                            
31640                                                

                                           for_tokenizer named_companies  \
23     i took stamps      and iac both not looking go...             IAC   
26                sold amd aug23rd call for a fat profit             AMD   
47     msft profits im still not selling it tho just ...            MSFT   
54                                        139 50 and ath             ATH   
56                                   139 50 and ath  wym             ATH   
...                                                  ...             ...   
31617                                edit is popping off            EDIT   
31623      oxy broke the resistance line on brad s chart             OXY   
31626  if pixy runs from being included in the benzin...            PIXY   
31637       have you done a video in gogo by any chance             GOGO   
31640                                             crm                CRM   

       isUpper  
23       False  
26       False  
47       False  
54       False  
56       False  
...        ...  
31617    False  
31623    False  
31626    False  
31637    False  
31640     True  

[6349 rows x 14 columns]

In [804]:
# Comments
# Trade Abbreviation terms
non_ticker_expressions = {
    "ath ": "all time High",
    "pdt": "pattern day trader",
    "rh": "RobinHood",
    "td": "TD Ameritrade", 
    "tos": "Think or Swim",
    "leg": "a trade",
    "pump": "overvalue",
    "exp": "expiration",
    "dd": "due diligence",
    "er": "earnings report"
}

In [9]:
# Comments
# Read SQL table
with sql.connect("../data/interim/discord/discord.db") as con:
    total_df_w_new_cols = pd.read_sql("SELECT * FROM comments", con=con)

In [10]:
# Comments
total_df_w_new_cols = total_df_w_new_cols.drop("pk", axis=1)

In [11]:
# Comments
total_df_w_new_cols.timestamp = pd.to_datetime(total_df_w_new_cols.timestamp)

In [15]:
# Comments
total_df_w_new_cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31934 entries, 0 to 31933
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   id           31934 non-null  object             
 1   name         31934 non-null  object             
 2   content      31934 non-null  object             
 3   timestamp    31934 non-null  datetime64[ns, UTC]
 4   channel      31934 non-null  object             
 5   server       31934 non-null  object             
 6   isBot        31934 non-null  int64              
 7   mentions     31934 non-null  object             
 8   emojis       31934 non-null  object             
 9   links        31934 non-null  object             
 10  chat_emotes  31934 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(9)
memory usage: 2.7+ MB


In [16]:
# Comments
total_df_w_new_cols

content  \
0                                     Play account today   
1      @everyone today’s total. This doesn’t calculat...   
2                                                  Crack   
3                                              Big crack   
4                                                 Heroin   
...                                                  ...   
31929  <@!644997307976122388> all of my shorter video...   
31930  <@428921853403463713> have you done a video in...   
31931               <@!184420431242067977> nope, not yet   
31932            they're the airline wifi company right?   
31933                                             CRM ??   

                             timestamp       channel server  isBot  \
0     2019-08-02 01:10:52.637000+00:00  wetlqd-ideas   Misc      0   
1     2019-08-02 01:13:47.923000+00:00  wetlqd-ideas   Misc      0   
2     2019-08-02 01:14:35.798000+00:00  wetlqd-ideas   Misc      0   
3     2019-08-02 01:14:39.682000+00:00  wetlqd-ideas   Misc      0   
4     2019-08-02 01:14:44.367000+00:00  wetlqd-ideas   Misc      0   
...                                ...           ...    ...    ...   
31929 2020-12-08 20:38:45.351000+00:00       trading    LSV      0   
31930 2020-12-08 20:39:17.246000+00:00       trading    LSV      0   
31931 2020-12-08 20:39:32.831000+00:00       trading    LSV      0   
31932 2020-12-08 20:39:38.597000+00:00       trading    LSV      0   
31933 2020-12-08 20:41:38.298000+00:00       options    LSV      0   

                 mentions emojis links chat_emotes  
0                                                   
1                everyone                           
2                                                   
3                                                   
4                                                   
...                   ...    ...   ...         ...  
31929  644997307976122388                           
31930  428921853403463713                           
31931  184420431242067977                           
31932                                               
31933                                               

[31934 rows x 11 columns]

In [20]:
# Comments
total_df_w_new_cols[lambda x: x.content.str.contains(company_regex)]

content  \
23     I took stamps .... and IAC both not looking go...   
26                Sold AMD aug23rd call for a fat profit   
47     MSFT profits im still not selling it tho just ...   
54                                       $139.50 and ATH   
56                                  $139.50 and ATH? Wym   
...                                                  ...   
31910                      @everyone EDIT is popping off   
31916     $OXY broke the resistance line on Brad's chart   
31919  if PIXY runs from being included in the Benzin...   
31930  <@428921853403463713> have you done a video in...   
31933                                             CRM ??   

                             timestamp       channel   server  isBot  \
23    2019-08-08 03:31:33.793000+00:00  wetlqd-ideas     Misc      0   
26    2019-08-08 13:33:00.968000+00:00  wetlqd-ideas     Misc      0   
47    2019-08-13 14:21:58.192000+00:00  wetlqd-ideas     Misc      0   
54    2019-08-13 19:01:16.234000+00:00  wetlqd-ideas     Misc      0   
56    2019-08-13 19:05:27.213000+00:00  wetlqd-ideas     Misc      0   
...                                ...           ...      ...    ...   
31910 2020-12-08 20:25:14.542000+00:00       trading      LSV      0   
31916 2020-12-08 20:26:51.453000+00:00       trading      LSV      0   
31919 2020-12-08 20:27:59.342000+00:00    trade-chat  Learn-n      0   
31930 2020-12-08 20:39:17.246000+00:00       trading      LSV      0   
31933 2020-12-08 20:41:38.298000+00:00       options      LSV      0   

                 mentions emojis links chat_emotes  
23                                                  
26                                                  
47                                                  
54                                                  
56                                                  
...                   ...    ...   ...         ...  
31910            everyone                           
31916                                               
31919                                               
31930  428921853403463713                           
31933                                               

[6483 rows x 11 columns]

In [25]:
# Comments
company_regex = re.compile(r"[A-Z]{2,4}")

In [26]:
# Comments
total_df_w_new_cols[lambda x: x.content.str.contains(company_regex)]

content  \
23     I took stamps .... and IAC both not looking go...   
26                Sold AMD aug23rd call for a fat profit   
47     MSFT profits im still not selling it tho just ...   
54                                       $139.50 and ATH   
56                                  $139.50 and ATH? Wym   
...                                                  ...   
31910                      @everyone EDIT is popping off   
31916     $OXY broke the resistance line on Brad's chart   
31919  if PIXY runs from being included in the Benzin...   
31930  <@428921853403463713> have you done a video in...   
31933                                             CRM ??   

                             timestamp       channel   server  isBot  \
23    2019-08-08 03:31:33.793000+00:00  wetlqd-ideas     Misc      0   
26    2019-08-08 13:33:00.968000+00:00  wetlqd-ideas     Misc      0   
47    2019-08-13 14:21:58.192000+00:00  wetlqd-ideas     Misc      0   
54    2019-08-13 19:01:16.234000+00:00  wetlqd-ideas     Misc      0   
56    2019-08-13 19:05:27.213000+00:00  wetlqd-ideas     Misc      0   
...                                ...           ...      ...    ...   
31910 2020-12-08 20:25:14.542000+00:00       trading      LSV      0   
31916 2020-12-08 20:26:51.453000+00:00       trading      LSV      0   
31919 2020-12-08 20:27:59.342000+00:00    trade-chat  Learn-n      0   
31930 2020-12-08 20:39:17.246000+00:00       trading      LSV      0   
31933 2020-12-08 20:41:38.298000+00:00       options      LSV      0   

                 mentions emojis links chat_emotes  
23                                                  
26                                                  
47                                                  
54                                                  
56                                                  
...                   ...    ...   ...         ...  
31910            everyone                           
31916                                               
31919                                               
31930  428921853403463713                           
31933                                               

[6483 rows x 11 columns]

In [131]:
# Comments
# Key Abbreviations that point to a trade, result or trading platform
trade_abbvs = {"ath ": "all time High",
 "pdt": "pattern day trader",
"rh": "RobinHood",
"td": "TD Ameritrade", 
"tos": "Think or Swim",
"leg": "a trade",
"pump": "overvalue",
"exp": "expiration",
"dd": "due diligence",
"er": "earnings report",
"pm": "pre-market", # opening
"ah": "after-hours", # trading hours,
"am": "after-market", # analogous to ah
"gg": "good game", # bot comment and normal internet positive abbv
"ta": "technical analysis",
"est": "eastern standard time", # stock engages timezones
"ipo": "intial public offering", # new stocks
"ev": "electric vehicles", # or expected value
"us": "United States",
"itm": "in the money", 
"otm": "out of the money",
}

In [30]:
# Comments
total_df = total_df_w_new_cols.copy(deep=True)

In [32]:
# Comments
companies_starter = add_specialized_token_column(company_regex, "content", "companies")

In [33]:
# Comments
companies_starter = companies_starter[lambda x: x.companies != ""]

In [39]:
# Comments
ticker_dict= {}

In [41]:
# Comments
# recount tickers
def getCompany(company_str):
    companies = company_str.split(",")
    for c in companies:
        if c not in ticker_dict.keys():
            ticker_dict[c] = 1
        else:
            ticker_dict[c]+=1
    return None

In [43]:
# Comments
companies_starter.companies.apply(getCompany)

23       None
26       None
47       None
54       None
56       None
         ... 
31910    None
31916    None
31919    None
31930    None
31933    None
Name: companies, Length: 6483, dtype: object

In [49]:
# Comments
company_counts = pd.DataFrame.from_dict(ticker_dict, "index", columns=["counts"]).reset_index().rename(columns={"index": "company"})

In [53]:
# Comments
company_counts = company_counts.sort_values("counts", ascending=False, ignore_index=True)

In [101]:
# Comments
company_counts.head(65)

,company,counts
0,SPY,167
1,TSLA,166
2,NIO,151
3,US,140
4,AMD,133
...,...,...
60,RKT,28
61,ES,27
62,JKS,25
63,COVI,25


In [103]:
# Comments
companies_starter[lambda x: x.companies.str.contains("ES")]

content  \
555                             Put that in TESTIMONIALS   
688                                                  YES   
1004                                  TWLO TO THE RESCUE   
1848                 YESSSSSSSSSS <@!560675627271979020>   
1881   THANKS, YALL ARE GOOD COACHES!! <a:5769_Jotaro...   
...                                                  ...   
31565  /ES and /NQ are flying. No idea why equities a...   
31854                                       AESE running   
31858  $AESE for anyone that was playing it just went...   
31867  <@!693425461774123068>, you get any starters i...   
31872  Didn’t you know <@396160546140979212> the stoc...   

                             timestamp       channel           server  isBot  \
555   2019-09-26 22:31:06.803000+00:00  wetlqd-ideas             Misc      0   
688   2019-10-09 14:31:25.385000+00:00  wetlqd-ideas             Misc      0   
1004  2019-11-11 19:36:50.705000+00:00  wetlqd-ideas             Misc      0   
1848  2020-02-19 19:44:07.028000+00:00  wetlqd-ideas             Misc      0   
1881  2020-02-22 05:28:56.993000+00:00  wetlqd-ideas             Misc      0   
...                                ...           ...              ...    ...   
31565 2020-12-08 16:03:05.514000+00:00       profits   Legacy Trading      0   
31854 2020-12-08 20:13:36.592000+00:00   day-trading  Templar Trading      0   
31858 2020-12-08 20:13:53.474000+00:00  main-trading  Templar Trading      0   
31867 2020-12-08 20:15:04.429000+00:00   day-trading  Templar Trading      0   
31872 2020-12-08 20:16:18.939000+00:00   day-trading  Templar Trading      0   

                 mentions emojis links chat_emotes  \
555                                                  
688                                                  
1004                                                 
1848   560675627271979020                            
1881                                                 
...                   ...    ...   ...         ...   
31565                                                
31854                                                
31858                                                
31867  693425461774123068                            
31872  396160546140979212                            

                            companies  
555                    TEST,IMON,IALS  
688                               YES  
1004              TWLO,TO,THE,RESC,UE  
1848                   YESS,SSSS,SSSS  
1881   THAN,KS,YALL,ARE,GOOD,COAC,HES  
...                               ...  
31565                           ES,NQ  
31854                            AESE  
31858                            AESE  
31867                            AESE  
31872                            AESE  

[114 rows x 12 columns]

In [132]:
# Comments
trade_terms = [k.upper().strip() for k in list(trade_abbvs.keys())]

In [134]:
# Comments
# Approx tickers w/o trade terms
company_counts[lambda x: ~(x.company.isin(trade_terms))].shape

(2334, 2)

In [135]:
# Comments
company_counts_real = company_counts[lambda x: ~(x.company.isin(trade_terms))]

In [137]:
# Comments
company_counts_real.to_csv("../data/interim/trading_counts.csv", index=False)

In [139]:
# Comments
# Intial Counts
company_counts_real

,company,counts
0,SPY,167
1,TSLA,166
2,NIO,151
4,AMD,133
5,AAPL,114
...,...,...
2348,NDS,1
2349,FRIE,1
2350,SAID,1
2351,BEIN,1


In [145]:
# Comments
# A few generic words are still here but looks like a good start list
company_counts_real.head(50)

,company,counts
0,SPY,167
1,TSLA,166
2,NIO,151
4,AMD,133
5,AAPL,114
9,DKNG,75
10,PLTR,74
11,ETF,73
13,PT,61
14,ACB,60


In [171]:
# Comments
companies_lowered = company_counts_real.company.apply(str.lower).values

In [184]:
# Comments
top100 = company_counts_real.head(100).company.values

In [185]:
# Comments
# Parser
hundred_dict = {k: "" for k in top100}

In [48]:
# Comments
# Initial c.c.
classifiable_companies[lambda x: x.Symbol == "WKHS"]

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,exchange
3032,WKHS,Workhorse Group Inc. Common Stock,$5.515,0.155,2.892%,860263391.0,United States,NaN,1917372,Capital Goods,Auto Manufacturing,NASDAQ


In [49]:
# Comments
# See which ones I can Get
classed_dict = {k: v for k, v in zip(classifiable_companies.Symbol.values, classifiable_companies.Name.values)}

In [51]:
# Comments
for k in hundred_dict.keys():
    try:
        hundred_dict[k] = classed_dict[k]
    except KeyError:
        hundred_dict[k] = ""

In [52]:
# Comments
empty_keys = []
for k, v in hundred_dict.items():
    if v == "":
        empty_keys.append(k)

In [53]:
# Comments
# Caught Tickers that are not listed (includes equity adjancent terms)
len(empty_keys)

27

In [54]:
# Comments
# Define ETFs, Equities that have been acquired and trade terms
hundred_dict['SPY'] = "S&P 500"
hundred_dict['ETF'] = "Exchange Traded Fund"
hundred_dict['BTC'] = "Bitcoin"
hundred_dict['VIX'] = "Volatility Index"
hundred_dict['SHLL'] = "Tortoise Acquistion"
hundred_dict['NQ'] = "Non-Qualified Options"
hundred_dict['MACD'] = "Moving Average Convergence Divergence"
hundred_dict['SQQQ'] = "Short QQQ ETF"
hundred_dict['WSB'] = "Wallstreet Bets"
hundred_dict['VWAP'] = "Volume-Weighted Average Trading"
hundred_dict['VXX'] = "Vix Short"
hundred_dict['APPL'] = "Apple Inc."
hundred_dict['QQQ'] = "Invesco QQQ Trust Shares"
hundred_dict['USD'] = "United States Dollar"
hundred_dict['IV'] = "Implied Volatility"
hundred_dict['GRAF'] = "VLDR"

In [55]:
# Comments
empty_keys = []
for k, v in hundred_dict.items():
    if v == "":
        empty_keys.append(k)

In [56]:
# Comments
len(empty_keys)

11

In [57]:
# Comments
# Can be purged, trading lingo such as SPAC=Specialized Trading Company, SL=Stop Loss
empty_keys

['LOL', 'SPAC', 'KEKW', 'TO', 'COVI', 'XD', 'FOMO', 'SL', 'FDA', 'OOOO', 'THE']

In [58]:
# Write to find the 100 companies mentions and their lowercase counterparts.

In [59]:
# Comments
# The 100 Trading Terms
hundred_dict

{'SPY': 'S&P 500',
 'TSLA': 'Tesla Inc. Common Stock',
 'NIO': 'NIO Inc. American depositary shares each  representing one Class A ordinary share',
 'AMD': 'Advanced Micro Devices Inc. Common Stock',
 'AAPL': 'Apple Inc. Common Stock',
 'DKNG': 'DraftKings Inc. Class A Common Stock',
 'PLTR': 'Palantir Technologies Inc. Class A Common Stock',
 'ETF': 'Exchange Traded Fund',
 'PT': 'Pintec Technology Holdings Limited American Depositary Shares',
 'ACB': 'Aurora Cannabis Inc. Common Shares',
 'ZM': 'Zoom Video Communications Inc. Class A Common Stock',
 'BABA': 'Alibaba Group Holding Limited American Depositary Shares each representing eight Ordinary share',
 'FB': 'Meta Platforms Inc. Class A Common Stock',
 'BTC': 'Bitcoin',
 'BA': 'Boeing Company (The) Common Stock',
 'VIX': 'Volatility Index',
 'WKHS': 'Workhorse Group Inc. Common Stock',
 'SQ': 'Square Inc. Class A Common Stock',
 'LOL': '',
 'BYND': 'Beyond Meat Inc. Common Stock',
 'NVDA': 'NVIDIA Corporation Common Stock',
 'IDEX

In [60]:
# Comments
# 
list(hundred_dict.keys())

['SPY',
 'TSLA',
 'NIO',
 'AMD',
 'AAPL',
 'DKNG',
 'PLTR',
 'ETF',
 'PT',
 'ACB',
 'ZM',
 'BABA',
 'FB',
 'BTC',
 'BA',
 'VIX',
 'WKHS',
 'SQ',
 'LOL',
 'BYND',
 'NVDA',
 'IDEX',
 'SPAC',
 'KEKW',
 'AMZN',
 'PFE',
 'TO',
 'MSFT',
 'NKLA',
 'GNUS',
 'FSLY',
 'SHLL',
 'PTON',
 'RSI',
 'ROKU',
 'PINS',
 'CRSR',
 'NQ',
 'PLUG',
 'WWR',
 'SPAQ',
 'HYLN',
 'SNDL',
 'JD',
 'SNAP',
 'RKT',
 'ES',
 'JKS',
 'COVI',
 'NNDM',
 'MACD',
 'GME',
 'SE',
 'IT',
 'MA',
 'SQQQ',
 'KO',
 'WSB',
 'XD',
 'FOMO',
 'MARA',
 'HEAR',
 'CRM',
 'CGC',
 'IZEA',
 'CBAT',
 'SL',
 'VWAP',
 'VXX',
 'APPL',
 'INTC',
 'SPCE',
 'AAL',
 'SPI',
 'ON',
 'AREC',
 'NFLX',
 'QQQ',
 'UONE',
 'BLNK',
 'HD',
 'ING',
 'FDA',
 'DIS',
 'MO',
 'OOOO',
 'THE',
 'TLRY',
 'ADTX',
 'YOU',
 'LMNL',
 'JMIA',
 'USD',
 'EXAS',
 'IV',
 'SOLO',
 'XERS',
 'DPW',
 'GILD',
 'GRAF']

In [61]:
# Comments
# Financial Terms to Keep 
terms_list = ['SPY', 'ETF' , 'BTC', 'VIX',
'NQ','MACD', 'SQQQ', 'WSB', 'VWAP','VXX', 
'APPL','QQQ','USD','IV'] 
#hundred_dict['SHLL'] = "Tortoise Acquistion"
#hundred_dict['GRAF'] = "VLDR"

In [62]:
# Comments
# See which to skip when calling to API
skip_keys = empty_keys + terms_list

In [230]:
# Comments
# get time bounds for calls
start_date = total_df_w_new_cols.timestamp[0].strftime("%Y-%m-%d")
end_date = (total_df_w_new_cols.timestamp[31933]+ dt.timedelta(1)).strftime('%Y-%m-%d') 

In [249]:
# Comments
end_date

'2020-12-09'

In [63]:
# Comments
y_tickers = [f for f in list(hundred_dict.keys()) if f not in skip_keys + empty_keys]

In [242]:
# Comments
# ETF test
spy_ticker = yf.Ticker('SPY')

In [244]:
# Comments
sample_df = yf.download('SPY', start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [261]:
# Comments
stock_movements_df = pd.DataFrame()

In [262]:
# Comments
# Call and Create a single DataFrame
for c in y_tickers:
    _temp_df = yf.download(c, start=start_date, end=end_date)
    _temp_df = _temp_df.assign(ticker=c).reset_index().rename(columns={"index": "date"})
    stock_movements_df = pd.concat([stock_movements_df, _temp_df], axis=0, ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [265]:
# Comments
stock_movements_df = stock_movements_df.rename(columns={"Adj Close": "Adj_Close"})

In [12]:
# Comments
start_date = total_df_w_new_cols.timestamp[0].strftime("%Y-%m-%d")

In [13]:
# Comments
end_date = (total_df_w_new_cols.timestamp[31933]+ dt.timedelta(1)).strftime('%Y-%m-%d') 

In [15]:
# Comments
# Amount of Days between first and last comment collected
total_df_w_new_cols.timestamp[0] - total_df_w_new_cols.timestamp[31933]

Timedelta('-495 days +04:29:14.339000')

In [21]:
# Comments
# Timedelta of trading window (9:30 to 4:00)
dt.datetime(2019, 8, 2, 16, 0, 0) - dt.datetime(2019, 8, 2, 9, 30, 0)

datetime.timedelta(seconds=23400)

In [28]:
# Comments
len(my_list)

8201

In [30]:
# Comments
my_list[1]

['2021-12-23 19:55:00', '130.6', '130.6', '130.6', '130.6', '103']

In [31]:
# Comments
my_list[-1]

['2021-11-29 04:01:00', '116.92', '116.92', '116.92', '116.92', '900']

In [32]:
# Comments 
# A minute to minute pull for equities
# CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year2month12&apikey={ALPHA_VANTAGE_API_KEY}'

# with requests.Session() as s:
#     download = s.get(CSV_URL)
#     decoded_content = download.content.decode('utf-8')
#     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
#     new_list = list(cr)

In [37]:
# Comments 
# OHLC from AlphaVantage
new_list[-1]

['2020-01-07 06:21:00',
 '116.163603611',
 '116.163603611',
 '116.163603611',
 '116.163603611',
 '104']

In [65]:
# Comments 
[i for i, x in enumerate(y_tickers) if x == "GRAF"]

[74]

In [67]:
# Comments 
# Remove GRAF
y_tickers.pop(74)

'GRAF'

In [68]:
# Comments
# Add renamed Ticker
y_tickers.append("VLDR")

In [81]:
# Comments
len(my_list)

8201

In [185]:
# Financial Data
# CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=IBM&interval=1min&slice=year2month6&apikey={ALPHA_VANTAGE_API_KEY}'

# with requests.Session() as s:
#     download = s.get(CSV_URL)
#     decoded_content = download.content.decode('utf-8')
#     cr = csv.reader(decoded_content.splitlines(), delimiter=',')
#     new_list = list(cr)

In [103]:
# Financial Data
new_list[-1]

['2020-07-06 04:01:00',
 '107.092161156',
 '107.092161156',
 '107.092161156',
 '107.092161156',
 '251']

In [105]:
# Financial Data
new_list[1]

['2020-08-03 18:34:00',
 '110.253912854',
 '110.253912854',
 '110.253912854',
 '110.253912854',
 '104']

In [106]:
# Financial Data
numbs = [6, 5, 4, 3, 2, 1]

In [180]:
# Financial Data
[i for i, x in enumerate(y_tickers) if x == "TSLA"]

[0]

In [247]:
# Financial Data
# f = 0
# for tick in ticker_set:
#     print(f"Starting {tick}")
#     for n in numbs:
#         CSV_URL = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol={tick}&interval=1min&slice=year2month{n}&apikey={ALPHA_VANTAGE_API_KEY}'
#         download = pd.read_csv(CSV_URL, sep=",", header=0, encoding="utf-8").assign(ticker = tick)    
#         with sql.connect("../data/interim/stocks.db") as con:
#             download.to_sql("minute", con=con, index=False, if_exists="append")
#         f += 1
#         if f % 5 == 0:
#             print(f"Added {f} Sleeping...")
#             time.sleep(68)
#     print(f"Added {tick}")

Starting CGC
Added 5 Sleeping...
Added CGC
Starting SOLO
Added 10 Sleeping...
Added SOLO
Starting VLDR
Added 15 Sleeping...
Added VLDR
Starting SPI
Added 20 Sleeping...
Added SPI
Starting LMNL
Added 25 Sleeping...
Added 30 Sleeping...
Added LMNL
Starting EXAS
Added 35 Sleeping...
Added EXAS
Starting INTC
Added 40 Sleeping...
Added INTC
Starting NFLX
Added 45 Sleeping...
Added NFLX
Starting DIS
Added 50 Sleeping...
Added DIS
Starting ADTX
Added 55 Sleeping...
Added 60 Sleeping...
Added ADTX
Starting JMIA
Added 65 Sleeping...
Added JMIA
Starting AAL
Added 70 Sleeping...
Added AAL
Starting DPW
Added 75 Sleeping...
Added DPW
Starting CBAT
Added 80 Sleeping...
Added CBAT


In [211]:
# Financial Data
# with sql.connect("../data/interim/stocks.db") as con:
#     download.to_sql("minute", con=con, index=False, if_exists="append")

0

In [248]:
# Financial Data
# with sql.connect("../data/interim/stocks.db") as con:
#     t = pd.read_sql("SELECT * FROM minute WHERE ticker = 'TSLA'", con=con)

In [232]:
# Financial Data
ticker_set = (set(y_tickers)).difference(set(t.ticker.unique()))

In [246]:
# Financial Data
ticker_set = ticker_set.difference({"ING", "TLRY", "MARA", "YOU", "UONE", "IZEA","AREC","BLNK","HD","SPCE","SHLL","GILD","MO","HEAR","CRM","XERS", 'TSLA', "ON"})

In [251]:
# Financial Data
t.time = pd.to_datetime(t.time)

In [252]:
# Financial Data
t.time.min()

Timestamp('2020-07-06 04:01:00')

In [253]:
# Financial Data
t.time.max()

Timestamp('2020-12-31 20:00:00')

In [257]:
# Financial Data
[hundred_dict[y] for y in y_tickers if y != "VLDR" else hundred_dict["GRAF"]]

SyntaxError: invalid syntax (<ipython-input-257-50bced56fd36>, line 1)

In [7]:
# Financial Data
news_dict = {'SPY': 'S&P 500',
 'TSLA': 'Tesla',
 'NIO': 'NIO',
 'AMD': 'Advanced Micro Devices',
 'AAPL': 'Apple',
 'DKNG': 'DraftKings',
 'PLTR': 'Palantir Technologies',
 'PT': 'Pintec Technology',
 'ACB': 'Aurora Cannabis',
 'ZM': 'Zoom',
 'BABA': 'Alibaba',
 'FB': 'Facebook',
 'BTC': 'Bitcoin',
 'BA': 'Boeing',
 'WKHS': 'Workhorse Group',
 'SQ': 'Square',
 'BYND': 'Beyond Meat',
 'NVDA': 'NVIDIA Corporation',
 'IDEX': 'Ideanomics',
 'AMZN': 'Amazon',
 'PFE': 'Pfizer',
 'MSFT': 'Microsoft',
 'NKLA': 'Nikola',
 'GNUS': 'Genius Brands',
 'FSLY': 'Fastly',
 'SHLL': 'Tortoise Acquistion',
 'PTON': 'Peloton',
 'RSI': 'Rush Street Interactive',
 'ROKU': 'Roku',
 'PINS': 'Pinterest',
 'CRSR': 'Corsair Gaming',
 'PLUG': 'Plug Power Inc.',
 'WWR': 'Westwater Resources',
 'SPAQ': 'Spartan Acquisition',
 'HYLN': 'Hyliion',
 'SNDL': 'Sundial Growers',
 'JD': 'JD.com',
 'SNAP': 'Snap Inc.',
 'RKT': 'Rocket Companies Inc.',
 'ES': 'Eversource Energy',
 'JKS': 'JinkoSolar',
 'NNDM': 'Nano Dimension',
 'GME': 'GameStop',
 'SE': 'Sea Limited',
 'IT': 'Gartner',
 'MA': 'Mastercard',
 'SQQQ': 'Short QQQ ETF',
 'KO': 'Coca-Cola Company',
 'MARA': 'Marathon Digital',
 'HEAR': 'Turtle Beach',
 'CRM': 'Salesforce',
 'CGC': 'Canopy Growth',
 'IZEA': 'IZEA Worldwide',
 'CBAT': 'CBAK Energy Technology Inc. Common Stock',
 'VXX': 'Vix Short',
 'APPL': 'Apple',
 'INTC': 'Intel',
 'SPCE': 'Virgin Galactic',
 'AAL': 'American Airlines',
 'SPI': 'SPI Energy',
 'ON': 'ON Semiconductor',
 'AREC': 'American Resources',
 'NFLX': 'Netflix',
 'QQQ': 'Invesco QQQ Trust Shares',
 'UONE': 'Urban One',
 'BLNK': 'Blink Charging',
 'HD': 'Home Depot',
 'ING': 'ING Group',
 'DIS': 'Disney',
 'MO': 'Altria',
 'TLRY': 'Tilray',
 'ADTX': 'Aditxt',
 'YOU': 'Clear Secure',
 'LMNL': 'Liminal BioSciences',
 'JMIA': 'Jumia Technologies',
 'EXAS': 'Exact Sciences',
 'SOLO': 'Electrameccanica Vehicles',
 'XERS': 'Xeris Biopharma',
 'DPW': 'Ault Global',
 'GILD': 'Gilead Sciences',
 'GRAF': 'Velodyne Lidar'}

In [8]:
# Financial Data
# Missing Keys
set(news_dict.keys()).difference(set(daily_data.ticker.unique()))

{'APPL', 'BTC', 'GRAF', 'QQQ', 'SHLL', 'SPAQ', 'SPY', 'SQQQ', 'VXX', 'YOU'}

In [60]:
# Financial Data
# pulled minute and daily data
with sql.connect("../data/interim/stocks.db") as con:
    daily_data = pd.read_sql("SELECT * FROM daily", con=con, index_col="pk", parse_dates={"Date": "%Y-%m-%d"})
    minute = pd.read_sql("SELECT * FROM minute WHERE ticker = 'TSLA'", con=con, parse_dates={"time": "%Y-%m-%d %H:%M:%S"})

In [57]:
# Financial Data
# Daily Companies
len(daily_data.ticker.unique())

71

In [79]:
# Financial Data
daily_data[lambda x: (x.ticker == "TSLA")]

,Date,Open,High,Low,Close,Adj_Close,Volume,ticker
pk,,,,,,,,
0,2019-08-02,46.270000,47.254002,45.846001,46.868000,46.868000,30682500.0,TSLA
1,2019-08-05,45.919998,46.273998,45.155998,45.664001,45.664001,35141500.0,TSLA
2,2019-08-06,46.375999,46.500000,45.150002,46.150002,46.150002,27821000.0,TSLA
3,2019-08-07,45.299999,46.714001,45.160000,46.683998,46.683998,23882500.0,TSLA
4,2019-08-08,46.889999,47.959999,46.529999,47.660000,47.660000,26371500.0,TSLA
...,...,...,...,...,...,...,...,...
337,2020-12-02,556.440002,571.539978,541.210022,568.820007,568.820007,47775700.0,TSLA
338,2020-12-03,590.020020,598.969971,582.429993,593.380005,593.380005,42552000.0,TSLA
339,2020-12-04,591.010010,599.039978,585.500000,599.039978,599.039978,29401300.0,TSLA


In [ ]:
# Financial Data
# Yahoo API 
for c in y_tickers:
    _temp_df = yf.download(c, start=start_date, end=end_date)
    _temp_df = _temp_df.assign(ticker=c).reset_index().rename(columns={"index": "date"})
    stock_movements_df = pd.concat([stock_movements_df, _temp_df], axis=0, ignore_index=True)

In [18]:
# Financial Data
# Yahoo API
vxx = yf.ticker.Ticker("VXX")
dl = vxx.history("5y", start="2019-01-01", end="2021-12-31")
vxx_info = vxx.info

In [19]:
# Financial Data
dl

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-01-02,193.880005,195.192001,181.759995,182.960007,513725,0,0.0
2019-01-03,186.399994,194.639999,185.160004,190.559998,254975,0,0.0
2019-01-04,183.240005,184.119995,174.919998,175.440002,326375,0,0.0
2019-01-07,174.119995,176.567993,169.399994,171.520004,640875,0,0.0
2019-01-08,168.559998,174.479996,167.520004,168.039993,236325,0,0.0
...,...,...,...,...,...,...,...
2021-12-23,20.200001,20.280001,19.950001,20.180000,31655100,0,0.0
2021-12-27,20.020000,20.290001,19.600000,19.700001,26696300,0,0.0
2021-12-28,19.590000,19.799999,19.170000,19.209999,30755200,0,0.0


In [9]:
# Financial Dagta
vxx_info

{'exchange': 'BTS',
 'shortName': 'iPath Series B S&P 500 VIX Shor',
 'longName': 'iPath Series B S&P 500 VIX Short-Term Futures ETN',
 'exchangeTimezoneName': 'America/New_York',
 'exchangeTimezoneShortName': 'EST',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '-18000000',
 'quoteType': 'ETF',
 'symbol': 'VXX',
 'messageBoardId': 'finmb_548515655',
 'market': 'us_market',
 'annualHoldingsTurnover': None,
 'enterpriseToRevenue': None,
 'beta3Year': 2.32,
 'profitMargins': None,
 'enterpriseToEbitda': None,
 '52WeekChange': None,
 'morningStarRiskRating': None,
 'forwardEps': None,
 'revenueQuarterlyGrowth': None,
 'fundInceptionDate': 1516147200,
 'annualReportExpenseRatio': None,
 'totalAssets': 821818112,
 'bookValue': None,
 'fundFamily': 'Milleis Investissements Funds',
 'lastFiscalYearEnd': None,
 'netIncomeToCommon': None,
 'trailingEps': None,
 'lastDividendValue': None,
 'SandP52WeekChange': None,
 'priceToBook': None,
 'nextFiscalYearEnd': None,
 'yield': 0,
 'mostRecen

In [131]:
# Financial Data
set(news_dict.keys()).difference(set(daily_data.ticker.unique()))

{'APPL', 'BTC', 'GRAF', 'QQQ', 'SHLL', 'SPAQ', 'SPY', 'SQQQ', 'VXX', 'YOU'}

In [202]:
# Financial Data
# ETFs and Other Unique Securities I have interest in and can query the API with
non_companies = ["BTC-USD", "QQQ","SPY", "VXX", "SQQQ"]

In [271]:
# Financial Data
# Grab Data and Calculate Volatility and Turnover via OHLC Data
# Turnover will be approx. b/c of buybacks and secondary offerings
def create_daily_data(ticker):
    tick = yf.ticker.Ticker(ticker)
    historical_data = tick.history("5y")
    outstanding = tick.info.get("sharesOutstanding")
    if outstanding == None:
        outstanding = 1
    daily_close = historical_data["Close"]
    pct_change = daily_close.pct_change().fillna(0)
    periods = 2
    # calc volatility
    vola = (pct_change.rolling(periods).std() * np.sqrt(periods)).fillna(0)
    historical_data = historical_data.assign(Volatility = vola)
    historical_data = historical_data.assign(Turnover = lambda x: x.Volume / outstanding)
    historical_data = historical_data.assign(company = ticker)
    return historical_data.reset_index().round({"Volatility": 6, "Turnover": 6})

In [165]:
# Financial Data
# list of companies
companies = list(daily_data.ticker.unique())

In [166]:
# Financial Data
# Delisted Companies
companies.append("VLDR")

In [ ]:
# Financial Data
# Full list of Securites to Analyze
companies = companies + non_companies

In [177]:
# Financial Data
for i, l in enumerate(companies):
    new_df = create_daily_data(l)
    full_daily_data = pd.concat([full_daily_data, new_df], axis=0, ignore_index=True)
    if i % 5 == 0:
        print(f"Finished with {i}")

Finished with 0
Finished with 5
Finished with 10
Finished with 15
Finished with 20
Finished with 25
Finished with 30
Finished with 35
Finished with 40


In [209]:
# Financial Data
full_daily_data = full_daily_data.rename(columns={"Stock Splits": "Stock_Splits"})

In [20]:
# Ananlyst Recommendations 
dis = yf.ticker.Ticker('DIS')
recommend_df = dis.recommendations

In [22]:
# Analyst Recommendations
recommend_df

,Firm,To Grade,From Grade,Action
Date,,,,
2012-02-08 08:43:00,Morgan Stanley,Overweight,,main
2012-02-08 13:09:00,UBS,Neutral,,main
2012-02-13 07:27:00,Miller Tabak,Buy,,main
2012-02-27 07:05:00,Goldman Sachs,Buy,,up
2012-03-20 06:03:00,Wunderlich,Hold,,main
...,...,...,...,...
2022-01-14 12:35:19,Guggenheim,Neutral,Buy,down
2022-02-07 13:47:41,Morgan Stanley,Overweight,,main
2022-02-10 13:29:00,JP Morgan,Overweight,,main


In [21]:
# Analyst Recommendations
# Firms Ratings Since 2020
recommend_df[lambda x: x.index < dt.datetime(2020, 12, 31)].value_counts('Firm')

Firm
Morgan Stanley            17
Credit Suisse             16
Citigroup                 13
UBS                       12
Imperial Capital          12
Nomura                    11
Wells Fargo               11
BMO Capital               11
Stifel Nicolaus           11
Barclays                  10
JP Morgan                  9
Rosenblatt                 8
Loop Capital               8
Deutsche Bank              8
Guggenheim                 7
FBR Capital                6
RBC Capital                6
Wunderlich                 6
B. Riley FBR               5
Topeka                     5
Pivotal Research           5
Macquarie                  5
Goldman Sachs              5
Atlantic Equities          4
Bernstein                  3
Hilliard Lyons             3
Bank of America            3
Cowen & Co.                3
PiperJaffray               2
Argus Research             2
KeyBanc                    2
BTIG Research              2
CLSA                       2
MoffettNathanson           2
Consumer 

In [23]:
# Analyst Recommendations
dis_info = dis.info

In [441]:
# Analyst Recommendations
dis_info

{'zip': '91521',
 'sector': 'Communication Services',
 'fullTimeEmployees': 152000,
 'longBusinessSummary': 'The Walt Disney Company, together with its subsidiaries, operates as an entertainment company worldwide. It operates through two segments, Disney Media and Entertainment Distribution; and Disney Parks, Experiences and Products. The company engages in the film and episodic television content production and distribution activities, as well as operates television broadcast networks under the ABC, Disney, ESPN, Freeform, FX, Fox, National Geographic, and Star brands; and studios that produces motion pictures under the Walt Disney Pictures, Twentieth Century Studios, Marvel, Lucasfilm, Pixar, and Searchlight Pictures banners. It also offers direct-to-consumer streaming services through Disney+, Disney+ Hotstar, ESPN+, Hulu, and Star+; sale/licensing of film and television content to third-party television and subscription video-on-demand services; theatrical, home entertainment, and 

In [442]:
# Analyst Recommendations
dis.get_institutional_holders()

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",137572834,2021-09-29,0.0757,23273196327
1,Blackrock Inc.,119795456,2021-09-29,0.0659,20265797291
2,State Street Corporation,71939331,2021-09-29,0.0396,12169976625
3,Morgan Stanley,35294697,2021-09-29,0.0194,5970803891
4,State Farm Mutual Automobile Insurance Co,34404214,2021-09-29,0.0189,5820160882
5,"FMR, LLC",29338716,2021-09-29,0.0161,4963230585
6,"Geode Capital Management, LLC",28284577,2021-09-29,0.0156,4784901891
7,Price (T.Rowe) Associates Inc,26497603,2021-09-29,0.0146,4482599499
8,Bank of America Corporation,25996713,2021-09-29,0.0143,4397863938
9,Northern Trust Corporation,24632835,2021-09-29,0.0136,4167136696


In [463]:
# Analyst Recommendations
dis.recommendations[lambda x: x.index >= dt.datetime(2020, 12, 1)]

,Firm,To Grade,From Grade,Action
Date,,,,
2020-12-01 16:11:33,UBS,Neutral,,main
2020-12-02 16:32:17,Citigroup,Buy,,main
2020-12-09 09:40:28,KeyBanc,Overweight,,init
2020-12-09 12:47:44,Wells Fargo,Overweight,Equal-Weight,up
2020-12-09 14:50:13,Morgan Stanley,Overweight,,main
2020-12-09 15:27:57,Credit Suisse,Outperform,,main
2020-12-10 11:55:42,Truist Securities,Buy,,init
2020-12-11 09:41:57,KeyBanc,Overweight,,main
2020-12-11 16:49:05,Credit Suisse,Outperform,,main


In [6]:
# News Articles
# Selenium Setup
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "eager"
dates = []
pat = re.compile("\?cid=\d+")

### Wanted Columns for Investing Sites
* ticker of company
* title of article
* category 
* content (article content)
* release_date
* provider / publisher
* url
* article_id

In [652]:
# News Articles
# Investing.com Windows
# ~May 2020 - May 2021 to match the busiest time periods of chat data
co_array= [{"company": "TSLA", "start": 330, "stop": 185, "url": "https://www.investing.com/equities/tesla-motors-news"},
{"company": "NIO", "start": 23, "stop":13,  "url": "https://www.investing.com/equities/nio-inc-news"},
{"company":"AMD", "start":26, "stop":14,  "url": "https://www.investing.com/equities/adv-micro-device-news"},
{"company":"AAPL","start": 375, "stop":350,  "url": "https://www.investing.com/equities/apple-computer-inc-news"}, 
{"company":"DKNG", "start":22, "stop":8,  "url": "https://www.investing.com/equities/diamond-eagle-acquisition-corp-news"},
{"company":"PLTR", "start":9, "stop":1,  "url": "https://www.investing.com/equities/palantir-technologies-inc-news"},
{"company":"ZM", "start":29, "stop":12,  "url": "https://www.investing.com/equities/zoom-video-communications-news"},
{"company":"BABA", "start":100, "stop":45,  "url": "https://www.investing.com/equities/alibaba-news"},
{"company": "HYLN","start":1 , "end": 1, "url":"https://www.investing.com/equities/tortoise-acquisition-corp-opinion"},
{"company": "LMNL","start": 1, "end": 1, "url":"https://www.investing.com/equities/prometic-life-sciences-inc.-opinion"},
{"company": "BTC-USD","start": 1000, "end": 450, "url":"https://www.investing.com/crypto/bitcoin/news/"},
{"company": "IDEX","start": 1, "end": 1, "url":"https://www.investing.com/equities/you-on-demand-holdings-inc-news"},
{"company": "ACB","start": 10, "stop": 1, "url":"https://www.investing.com/equities/aurora-cannabis-news?cid=1055316"},
{"company": "CGC","start": 6, "stop": 1, "url":"https://www.investing.com/equities/tweed-marijuana-inc-news?cid=1057241"},
{"company": "IZEA","start": 1, "stop": 1, "url":"https://www.investing.com/equities/izea-inc-news"},
{"company": "CBAT","start": 1, "stop": 1, "url":"https://www.investing.com/equities/china-bak-battery-news"},
{"company":"BA", "start":160, "stop":50,  "url": "https://www.investing.com/equities/boeing-co-news"},
{"company":"WKHS", "start":3, "stop":1,  "url": "https://www.investing.com/equities/amp-holding-inc-news"},
{"company":"PFE", "start":250, "stop":238,  "url": "https://www.investing.com/equities/pfizer-news"},
{"company":"FB", "start":404, "stop":130, "url": "https://www.investing.com/equities/facebook-inc-news"},
{"company":"SQ", "start":20, "stop":7,  "url": "https://www.investing.com/equities/square-inc-news"},
{"company":"BYND", "start":21, "stop":8,  "url": "https://www.investing.com/equities/beyond-meat-inc-news"} ,
{"company":"NVDA", "start":52, "stop":23,  "url": "https://www.investing.com/equities/nvidia-corp-news"},
{"company":"AMZN", "start":335, "stop":155, "url": "https://www.investing.com/equities/amazon-com-inc-news"},
{"company": "MSFT","start": 220, "stop": 70, "url":"https://www.investing.com/equities/microsoft-corp-news"},
{"company":"NKLA", "start":13, "stop":3, "url": "https://www.investing.com/equities/nikola-corp-news/"},
{"company":"PTON", "start":30, "stop":12, "url":"https://www.investing.com/equities/peloton-interactive-inc-news"},
{"company":"ROKU", "start":16, "stop":7,  "url": "https://www.investing.com/equities/roku-news"},
{"company":"PINS", "start":15, "stop":7,  "url": "https://www.investing.com/equities/pinterest-inc-news"},
{"company": "DIS", "start": 57, "stop": 25, "url": "https://www.investing.com/equities/disney-news"},
{"company": "GNUS","start": 2, "stop": 1, "url":"https://www.investing.com/equities/genius-brands-intl.-news"},
{"company": "FSLY","start": 5, "stop": 1, "url":"https://www.investing.com/equities/fastly-inc-news"},
{"company": "RSI","start": 1, "stop": 1, "url":"https://www.investing.com/equities/dmy-technology-group-news"},
{"company": "CRSR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/corsair-gaming-inc-news"},
{"company": "PLUG","start": 10, "stop": 1, "url":"https://www.investing.com/equities/plug-power-news"},
{"company": "WWR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/uranium-resources-news"},
{"company": "SNDL","start": 5, "stop": 1, "url":"https://www.investing.com/equities/sundial-growers-inc-news"},
{"company": "JD","start": 35, "stop": 15, "url":"https://www.investing.com/equities/jd.com-inc-adr-news"},
{"company": "SNAP","start": 28, "stop": 10, "url":"https://www.investing.com/equities/snap-inc-news"},
{"company": "RKT","start": 2, "stop": 1, "url":"https://www.investing.com/equities/rocket-companies-inc-news"},
{"company": "ES","start": 3, "stop": 1, "url":"https://www.investing.com/equities/northeast-utilities-news"},
{"company": "JKS","start": 3, "stop": 1, "url":"https://www.investing.com/equities/jinkosolar-holding-comp-ltd-news"},
{"company": "NNDM","start": 2, "stop": 1, "url":"https://www.investing.com/equities/nano-dimension-ltd-news"},
{"company": "GME","start": 99, "stop": 20, "url":"https://www.investing.com/equities/gamestop-corp-news"},
{"company": "SE","start": 7, "stop": 1, "url":"https://www.investing.com/equities/sea-limited-news"},
{"company": "IT","start": 10, "stop": 1, "url":"https://www.investing.com/equities/gartner-news"},
{"company": "MA","start": 40, "stop": 10, "url":"https://www.investing.com/equities/mastercard-cl-a-news"},
{"company": "KO","start": 38, "stop": 8, "url":"https://www.investing.com/equities/coca-cola-co-news"},
{"company": "MARA","start": 4, "stop": 2, "url":"https://www.investing.com/equities/marathon-pa-news"},
{"company": "HEAR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/parametric-sound-corp-news"},
{"company": "CRM","start": 38, "stop": 12, "url":"https://www.investing.com/equities/salesforce-com-news"},
{"company": "IZEA","start": 1, "stop": 1, "url":"https://www.investing.com/equities/izea-inc-news"},
{"company": "CBAT","start": 1, "stop": 1, "url":"https://www.investing.com/equities/china-bak-battery-news"},
{"company": "INTC","start": 58, "stop": 20, "url":"https://www.investing.com/equities/intel-corp-news"},
{"company": "SPCE","start": 16, "stop": 1, "url":"https://www.investing.com/equities/social-capital-hedosophia-news"},
{"company": "AAL","start": 75, "stop": 23, "url":"https://www.investing.com/equities/american-airlines-group-news"},
{"company": "SPI","start": 1, "stop": 1, "url":"https://www.investing.com/equities/spi-energy-co-ltd-news"},
{"company": "ON","start": 2, "stop": 1, "url":"https://www.investing.com/equities/on-semiconductor-news"},
{"company": "AREC","start": 2, "stop": 1, "url":"https://www.investing.com/equities/american-resources-news"},
{"company": "NFLX","start": 92, "stop": 31, "url":"https://www.investing.com/equities/netflix,-inc.-news"},
{"company": "UONE","start": 1, "stop": 1, "url":"https://www.investing.com/equities/radio-one-(a)-news"},
{"company": "BLNK","start": 4, "stop": 3, "url":"https://www.investing.com/equities/car-charging-group-news"},
{"company": "HD","start": 28, "stop": 11, "url":"https://www.investing.com/equities/home-depot-news"},
{"company": "ING","start": 7, "stop": 2, "url":"https://www.investing.com/equities/ing-group-nv-news"},
{"company": "MO","start": 10, "stop": 1, "url":"https://www.investing.com/equities/altria-group-news"},
{"company": "TLRY","start": 11, "stop": 4, "url":"https://www.investing.com/equities/tilray-inc-news"},
{"company": "ADTX","start": 1, "stop": 1, "url":"https://www.investing.com/equities/aditx-therapeutics-inc-news"},
{"company": "JMIA","start": 1, "stop": 1, "url":"https://www.investing.com/equities/jumia-technologies-ag-news"},
{"company": "EXAS","start": 3, "stop": 1, "url":"https://www.investing.com/equities/exact-sciences-co-news"},
{"company": "SOLO","start": 1, "stop": 1, "url":"https://www.investing.com/equities/electrameccanica-vehicles-news"},
{"company": "XERS","start": 1, "stop": 1, "url":"https://www.investing.com/equities/xeris-pharmaceuticals-news"},
{"company": "DPW","start": 13, "stop": 4, "url":"https://www.investing.com/equities/deutsche-post-news"},
{"company": "GILD","start": 30, "stop": 6, "url":"https://www.investing.com/equities/gilead-sciences-inc-news"},
{"company": "VLDR","start": 1, "stop": 1, "url":"https://www.investing.com/equities/graf-industrial-corp-news"},
{"company": "QQQ","start": 9, "stop": 1, "url":"https://www.investing.com/etfs/powershares-qqqq-news"},
{"company": "SPY","start": 190, "stop": 5, "url":"https://www.investing.com/etfs/spdr-s-p-500-news"},
{"company": "VXX","start": 1, "stop": 1, "url":"https://www.investing.com/etfs/ipath-series-b-sp500-vix-st-futures-news"},
{"company": "SQQQ","start": 1, "stop": 1, "url":"https://www.investing.com/etfs/ultrapro-short-qqq-news"}]

In [88]:
# News Articles
# Scraper
driver = webdriver.Chrome("../../../Python/scraping/chromedriver2.exe", desired_capabilities=caps)
i = 0
test_df = pd.DataFrame()
for c in co_array:
    base_url = c["url"]
    ending_url = ""
    if re.findall(pat, c["url"]) != []:
        ending_url = re.findall(pat, base_url)[0]
        base_url = re.sub(pat, "", base_url)
    starting_url = base_url + "/" + str(c["stop"]) + ending_url
    
    ticker = c["company"]
    shuffle = False
    if (c["start"] - c["stop"]) >= 100:
        shuffle = True
    elif c["start"] == c["stop"]:
        c["start"] = 2
    for k in range(c["stop"], c["start"], 1): # c["start"] - c["stop"]
        driver.get(starting_url)
        main_window = driver.window_handles[0]
        dater = driver.find_elements(By.CSS_SELECTOR, "section#leftColumn > div.mediumTitle1")
        
        for d in dater:
            articles = d.find_elements(By.TAG_NAME, "article")
            if shuffle:
                random.shuffle(articles)
                articles = articles[:3]

            for art in articles:    
                id = art.get_attribute("data-id")
                link = art.find_element(By.TAG_NAME, "a").get_attribute("href") # title #leftColumn > div.mediumTitle1 > article:nth-child(2) > div.textDiv > p
                details = art.find_element(By.TAG_NAME, "div")
                title = art.find_element(By.CLASS_NAME, "title").text
                if title == "":
                    next
                for j, s in enumerate(details.find_elements(By.TAG_NAME, "span")):
                    if j == 1:
                        publisher = s.text.replace("By", "").strip()
                    elif j == 2:
                        date = s.text.replace("-", "").strip()
                link = art.find_element(By.TAG_NAME, "a").get_attribute("href")
                if link.startswith("https://www.investing.com"):
                    driver.execute_script(f"window.open(\"{link}\",\"_blank\");")
                    driver.switch_to.window(window_name=driver.window_handles[1])
                    artsy = driver.find_element(By.CSS_SELECTOR, "div.articlePage").text
                    driver.close()
                    driver.switch_to.window(window_name=driver.window_handles[0])
                    test_df.loc[i, ["id", "title", "link", "date", "publisher", "article", "ticker"]] = [id, title, link, date, publisher, artsy, ticker]
                i += 1
                
        starting_url = base_url + "/" + str(k) + ending_url
        time.sleep(4)
    print(f"finished with {ticker}")
driver.quit()

ipykernel_launcher:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


finished with MSFT
finished with NKLA
finished with PTON
finished with ROKU
finished with PINS
finished with DIS
finished with GNUS
finished with FSLY
finished with RSI
finished with CRSR
finished with PLUG
finished with WWR
finished with SNDL
finished with JD
finished with SNAP
finished with RKT
finished with ES
finished with JKS
finished with NNDM
finished with GME
finished with SE
finished with IT
finished with MA
finished with KO
finished with MARA
finished with HEAR
finished with CRM
finished with IZEA
finished with CBAT
finished with INTC
finished with SPCE
finished with AAL
finished with SPI
finished with ON
finished with AREC
finished with NFLX
finished with UONE
finished with BLNK
finished with HD
finished with ING
finished with MO
finished with TLRY
finished with ADTX
finished with JMIA
finished with EXAS
finished with SOLO
finished with XERS
finished with DPW
finished with GILD
finished with VLDR
finished with QQQ
finished with SPY
finished with VXX
finished with SQQQ


In [83]:
# News Articles
driver.quit()

In [37]:
# News Articles
# 
test_df

,id,title,link,date,publisher,article,ticker
0,2467735,Tesla tells China car cameras not activated ou...,https://www.investing.com/news/economy/tesla-s...,"Apr 07, 2021",Reuters,© Reuters. FILE PHOTO: Logo of the electric ve...,TSLA
1,2469136,Tesla lashes out at German red tape ahead of p...,https://www.investing.com/news/stock-market-ne...,"Apr 08, 2021",Reuters,© Reuters. FILE PHOTO: View of the Tesla Gigaf...,TSLA
2,2467806,"Sun, sea and cybernauts: the long road for Gre...",https://www.investing.com/news/technology-news...,"Apr 07, 2021",Reuters,© Reuters. Rowena Harding works at the garden ...,TSLA
3,2467802,CEO Mary Barra bets GM can grow beyond cars an...,https://www.investing.com/news/stock-market-ne...,"Apr 07, 2021",Reuters,© Reuters. FILE PHOTO: A mobile phone displays...,TSLA
4,2467806,"Sun, sea and cybernauts: the long road for Gre...",https://www.investing.com/news/technology-news...,"Apr 07, 2021",Reuters,© Reuters. Rowena Harding works at the garden ...,TSLA
...,...,...,...,...,...,...,...
11864,1759417,Stocks lower ahead of action-packed week,https://www.investing.com/news/stock-market-ne...,"Jan 28, 2019",Seeking Alpha,© Reuters. Stocks lower ahead of action-packed...,SQQQ
11865,1748359,"Stocks weather Brexit debacle, pound steadies",https://www.investing.com/news/stock-market-ne...,"Jan 16, 2019",Seeking Alpha,"© Reuters. Stocks weather Brexit debacle, poun...",SQQQ
11866,1746970,Equities boosted by China stimulus announcement,https://www.investing.com/news/stock-market-ne...,"Jan 15, 2019",Seeking Alpha,© Reuters. Equities boosted by China stimulus ...,SQQQ
11867,1745753,"Chinese trade data knocks Asia, U.S. futures",https://www.investing.com/news/stock-market-ne...,"Jan 14, 2019",Seeking Alpha,"© Reuters. Chinese trade data knocks Asia, U.S...",SQQQ


In [91]:
# News Articles
test_df.to_csv("../data/interim/articlesCK.csv", index_label="pk")

In [39]:
# News Articles
re.sub(re.compile(".+\(Reuters\) - "),"",  test_df.loc[11202, "article"].replace("\n", ""))

"Gilead Sciences Inc (NASDAQ:GILD) on Thursday said its first-quarter product sales rose 16%, including $1.46 billion in sales of its COVID-19 antiviral drug remdesivir, but sales of flagship HIV and hepatitis C drugs were down due to the effects of the pandemic.The biotech company reported adjusted earnings of $2.08 per share, edging out the average analyst estimate of $2.07 as compiled by Refinitiv. Net income rose to $1.72 billion from $1.54 billion.The 16% rise in revenue for the quarter to $6.4 billion was still short of Wall Street estimates of $6.73 billion.Gilead said the COVID-19 pandemic continued to hurt sales of its treatments for hepatitis C and HIV, due to fewer people going to their doctors during the pandemic. It now expects a more gradual recovery in the market starting in the current quarter.Excluding remdesivir, its product sales fell 11% to $4.9 billion.Sale of remdesivir, which is sold under the brand name Veklury, were also below analysts' estimates of $1.56 billi

In [243]:
# News Articles
# Publishers
test_df.groupby("publisher").count().sort_values("id", ascending=False)#.id.describe()

,id,title,link,date,article,ticker
publisher,,,,,,
Reuters,5644,5644,5644,5644,5644,5644
Investing.com,4868,4868,4868,4868,4868,4868
Cointelegraph,400,400,400,400,400,400
Bloomberg,385,385,385,385,385,385
StockNews,334,334,334,334,334,334
DailyCoin,93,93,93,93,93,93
BTC Peers,67,67,67,67,67,67
CoinQuora,47,47,47,47,47,47
Seeking Alpha,17,17,17,17,17,17


In [40]:
# News Articles
# sample article
sample_text_art = test_df.loc[4152, "article"].replace("\n", " ")#.replace("© Reuters. FILE PHOTO: American Airlines passenger planes crowd a runway where they are parked due to flight reductions to slow the spread of coronavirus disease (COVID-19), at Tulsa International Airport in Tulsa, Oklahoma, U.S. March 23, 2020. REUTERS/Nick Oxford/ By David Shepardson WASHINGTON (Reuters) -", "")

In [41]:
# News Articles
sample_text_art

'© Reuters. By Yasin Ebrahim Investing.com – The Dow moved off lows on Thursday, as big tech cut some of its losses but still remained deep in the red as investors digested economic data pointing to signs of a slowdown in the recovery at a time when hopes fade for further government stimulus. The Dow Jones Industrial Average fell 0.63%, or 177 points. The S&P 500 was down 0.63% while the Nasdaq Composite slipped 1.56%. Weaker-than-expected labor and housing market raised concerns that the pace of the recovery could be on the wane, prompting investors to rein in their bullish bets on stocks. Initial jobless claims came in at 860,000 on a seasonally adjusted basis in the week ended Sept. 12, the Department of Labor said Thursday. That was below the 893,000 reading from the prior week, but missing economists\' forecast for a reading of 850,000. "The downward trend in regular state initial jobless claims - that is, the headline number - has slowed to a crawl in the past couple of weeks," P

In [43]:
# News Articles
test_df.loc[test_df.publisher == "Reuters - Sep 08, 2020 5", "date"]

1879    By Reuters
Name: date, dtype: object

In [44]:
# News Articles 
test_df.loc[1879, :]

id                                                     2289328
title        Amazon's Bezos tops Forbes richest list, Trump...
link         https://www.investing.com/news/general/amazons...
date                                                By Reuters
publisher                             Reuters - Sep 08, 2020 5
article      For the third year in a row, Amazon (NASDAQ:AM...
ticker                                                      ZM
Name: 1879, dtype: object

In [45]:
# News Articles 
test_df.publisher.unique()

array(['Reuters', 'Investing.com', 'DailyCoin', 'BTC Peers', 'CoinQuora',
       'Cointelegraph', 'Bloomberg', 'StockNews',
       'Reuters - Jul 29, 2020', 'TipRanks', 'Reuters - Sep 08, 2020 5',
       'Seeking Alpha', 'Reuters - Jan 28, 2021',
       'Reuters - Oct 22, 2020'], dtype=object)

In [46]:
# News Articles 
test_df.loc[test_df.publisher == "Reuters - Sep 08, 2020 5", "date"] = "Sep 08, 2020"
test_df.loc[test_df.publisher == "Reuters - Sep 08, 2020 5", "publisher"] = "Reuters"
test_df.loc[test_df.publisher == "Reuters - Jan 28, 2021", "date"] = "Jan 28, 2021"
test_df.loc[test_df.publisher == "Reuters - Jan 28, 2021", "publisher"] = "Reuters"
test_df.loc[test_df.publisher == "Reuters - Jul 29, 2020", "date"] = "Jul 29, 2020"
test_df.loc[test_df.publisher == "Reuters - Jul 29, 2020", "publisher"] = "Reuters"
test_df.loc[test_df.publisher == "Reuters - Oct 22, 2020", "date"] = "Oct 22, 2020"
test_df.loc[test_df.publisher == "Reuters - Oct 22, 2020", "publisher"] = "Reuters"

In [47]:
# News Articles 
test_df.publisher.unique()

array(['Reuters', 'Investing.com', 'DailyCoin', 'BTC Peers', 'CoinQuora',
       'Cointelegraph', 'Bloomberg', 'StockNews', 'TipRanks',
       'Seeking Alpha'], dtype=object)

In [49]:
# News Articles
test_df[lambda x: x.publisher == 'Reuters']

,id,title,link,date,publisher,article,ticker
0,2467735,Tesla tells China car cameras not activated ou...,https://www.investing.com/news/economy/tesla-s...,"Apr 07, 2021",Reuters,© Reuters. FILE PHOTO: Logo of the electric ve...,TSLA
1,2469136,Tesla lashes out at German red tape ahead of p...,https://www.investing.com/news/stock-market-ne...,"Apr 08, 2021",Reuters,© Reuters. FILE PHOTO: View of the Tesla Gigaf...,TSLA
2,2467806,"Sun, sea and cybernauts: the long road for Gre...",https://www.investing.com/news/technology-news...,"Apr 07, 2021",Reuters,© Reuters. Rowena Harding works at the garden ...,TSLA
3,2467802,CEO Mary Barra bets GM can grow beyond cars an...,https://www.investing.com/news/stock-market-ne...,"Apr 07, 2021",Reuters,© Reuters. FILE PHOTO: A mobile phone displays...,TSLA
4,2467806,"Sun, sea and cybernauts: the long road for Gre...",https://www.investing.com/news/technology-news...,"Apr 07, 2021",Reuters,© Reuters. Rowena Harding works at the garden ...,TSLA
...,...,...,...,...,...,...,...
11849,2189894,Financing hunt during pandemic lifts May U.S. ...,https://www.investing.com/news/economic-indica...,"Jun 02, 2020",Reuters,© Reuters. FILE PHOTO: A number of grounded So...,SPY
11855,2183651,S&P 500 rises on boost from bank shares,https://www.investing.com/news/stock-market-ne...,"May 27, 2020",Reuters,© Reuters. New York Stock Exchange opens durin...,SPY
11856,2335122,Pandemic's latest spread triggers surge in VIX...,https://www.investing.com/news/stock-market-ne...,"Oct 28, 2020",Reuters,© Reuters. FILE PHOTO: FILE PHOTO: Statue of G...,VXX
11857,2093700,Investors piled into volatility bets before ma...,https://www.investing.com/news/stock-market-ne...,"Feb 25, 2020",Reuters,By Saqib Iqbal Ahmed\nNEW YORK (Reuters) - Fea...,VXX


In [50]:
# News Articles
# Articles still retain some nonstory metadata
test_df[lambda x: x.publisher == 'Reuters'].loc[0, "article"]

'© Reuters. FILE PHOTO: Logo of the electric vehicle maker Tesla is seen near a shopping complex in Beijing\nSHANGHAI (Reuters) -Cameras in Tesla (NASDAQ:TSLA) cars are not activated outside of North America, the U.S. automaker said on its Chinese social media page on Wednesday, seeking to assuage security concerns in the world\'s biggest car market.\nTesla faces scrutiny in China where the military in March banned Tesla cars from entering its complexes, citing security concerns over cameras in its vehicles, sources told Reuters.\n"Even in the United States, car owners can freely choose whether to turn on its (the camera system\'s) use. Tesla is equipped with a network security system with world-leading security levels to ensure user privacy protection," the electric carmaker wrote on Weibo (NASDAQ:WB), China\'s Twitter-like social media site.\nAt a virtual forum in Beijing in March, held not long after reports of the ban surfaced, Tesla founder Elon Musk emphasised the company\'s busi

In [53]:
# News Articles
for j in test_df.publisher.unique():
    print("="*30)
    print(j)
    print("="*30)
    print(test_df[lambda x: x.publisher == j].iloc[0, -2])

Reuters
© Reuters. FILE PHOTO: Logo of the electric vehicle maker Tesla is seen near a shopping complex in Beijing
SHANGHAI (Reuters) -Cameras in Tesla (NASDAQ:TSLA) cars are not activated outside of North America, the U.S. automaker said on its Chinese social media page on Wednesday, seeking to assuage security concerns in the world's biggest car market.
Tesla faces scrutiny in China where the military in March banned Tesla cars from entering its complexes, citing security concerns over cameras in its vehicles, sources told Reuters.
"Even in the United States, car owners can freely choose whether to turn on its (the camera system's) use. Tesla is equipped with a network security system with world-leading security levels to ensure user privacy protection," the electric carmaker wrote on Weibo (NASDAQ:WB), China's Twitter-like social media site.
At a virtual forum in Beijing in March, held not long after reports of the ban surfaced, Tesla founder Elon Musk emphasised the company's busin

In [ ]:
# News Articles
# Articles can have article, publisher and author at the start of the headlines since investing.com is pulling data from a few different sites.
# So they need to be clean with regeular expressions
    # .+ (Reuters) -  x
    # Investing.com - | -- x
    # (Bloomberg) -- x
    # StockNews => .+\n | copyright Reuters + Title\n
    # DailyCoin => Title\n
    # BTC Peers => Title\n
    # CoinQuora => Title\n
    # Seeking Alpha => Title\n
    # copyright Reuters + Title\n

In [55]:
# News Articles
# Hard coded names
b = -1
r_regex = re.compile(r".*\(Reuters\) -")
iv_regex = re.compile(r".*Investing.com (–|--|-)")
bloom_regex = re.compile(r".*\(Bloomberg\) --")

In [54]:
# News Articles
# Regex, title and any trailing data will be removed 
def cleanArticle(article, title, index):
    global b
    b += 1
    i = index[b]
    article = article.replace("\n", " ").strip()
    title = title[i]
    article = re.sub(re.compile(r".*" + title), "", article)
    article = re.sub(r_regex, "", article)
    article = re.sub(iv_regex, "", article)
    article = re.sub(bloom_regex, "", article)
    return article.strip()

In [644]:
# News Articles
test_df = test_df.assign(articles2 = lambda x: x.article.apply(cleanArticle, title=x.title, index=x.article.index))

In [650]:
# News Articles
test_df.drop_duplicates().reset_index(drop=True).drop("article", axis=1).rename(columns={"articles2": "article"}).to_csv("../data/interim/articlesCK.csv", index_label="pk")

In [657]:
# News Articles
test_df.date = pd.to_datetime(cleaned_articles.date)

In [670]:
# News Articles
test_df.sort_values(["ticker", "date"]).groupby("ticker").agg(["first", "last", "count"]).sort_values(["count", 'first'], ascending=[False, True])

In [56]:
# Analyst Recommendations
recommend_df.groupby(["Firm", "To Grade"]).count().sort_index(level=0).tail(20)

From Grade  Action
Firm              To Grade                          
RBC Capital       Sector Perform           3       3
                  Top Pick                 1       1
Rosenblatt        Buy                      8       8
                  Neutral                  1       1
Sterne Agee       Neutral                  1       1
Stifel Nicolaus   Buy                     10      10
                  Hold                     1       1
Tigress Financial Buy                      2       2
Topeka            Buy                      2       2
                  Hold                     2       2
                  Neutral                  1       1
Truist Securities Buy                      3       3
UBS               Buy                      9       9
                  Neutral                  5       5
Wells Fargo       Equal-Weight             2       2
                  Market Perform           1       1
                  Outperform               4       4
                  Overweight               8       8
Wunderlich        Buy                      1       1
                  Hold                     5       5

In [267]:
# Financial Data
# Vanguard GICS Sectors will provide baselines to compare portfolio to
gics_sectors = {"Communication Services ETF":"VOX",
"Consumer Discretionary ETF":"VCR",
"Consumer Staples":	"VDC",
"Energy ETF": "VDE",
"Financials ETF": "VFH",
"Health Care ETF": "VHT",
"Industrials ETF": "VIS",
"Information Technology ETF": "VGT",
"Materials ETF" : "VAW",
"Real Estate ETF": "VNQ",
"Utilities ETF": "VPU"}

In [279]:
# Financial Data
full_daily_data = pd.DataFrame()

In [282]:
# Financial Data
for i, l in enumerate(gics_sectors.values()):
    new_df = create_daily_data(l)
    full_daily_data = pd.concat([full_daily_data, new_df], axis=0, ignore_index=True)
    print(f"Finished with {l}")

Finished with VOX
Finished with VCR
Finished with VDC
Finished with VDE
Finished with VFH
Finished with VHT
Finished with VIS
Finished with VGT
Finished with VAW
Finished with VNQ
Finished with VPU


In [294]:
# Financial Data
full_daily_data = full_daily_data.rename(columns={"Stock Splits": "Stock_Splits"})

In [17]:
# Financial Data
with sql.connect("../data/interim/stocks.db") as con:
    company_dailys = pd.read_sql("SELECT * FROM daily", con=con)

In [291]:
# Financial Data
full_daily_data.index.name = "pk"

In [305]:
# Financial Data
daily_with_gics = pd.concat([company_dailys, full_daily_data], axis=0, ignore_index=True)

In [327]:
# Financial Data
daily_with_gics["Date"] = pd.to_datetime(daily_with_gics["Date"])

In [343]:
# Financial Data
company_dailys[lambda x: x.company == "AMZN"].drop("pk", axis=1)

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Volatility,Turnover,company
17774,2017-01-03 00:00:00,757.919983,758.760010,747.700012,753.669983,3521100,0.0,0.0,0.0000,0.0070,AMZN
17775,2017-01-04 00:00:00,758.390015,759.679993,754.200012,757.179993,2510500,0.0,0.0,0.0047,0.0050,AMZN
17776,2017-01-05 00:00:00,761.549988,782.400024,760.260010,780.450012,5830100,0.0,0.0,0.0261,0.0116,AMZN
17777,2017-01-06 00:00:00,782.359985,799.440002,778.479980,795.989990,5986200,0.0,0.0,0.0108,0.0119,AMZN
17778,2017-01-09 00:00:00,798.000000,801.770020,791.770020,796.919983,3446100,0.0,0.0,0.0187,0.0068,AMZN
...,...,...,...,...,...,...,...,...,...,...,...
19028,2021-12-27 00:00:00,3420.739990,3458.860107,3384.310059,3393.389893,2934400,0.0,0.0,0.0084,0.0058,AMZN
19029,2021-12-28 00:00:00,3403.649902,3443.520020,3382.709961,3413.219971,2731900,0.0,0.0,0.0140,0.0054,AMZN
19030,2021-12-29 00:00:00,3416.800049,3424.239990,3372.010010,3384.020020,1787700,0.0,0.0,0.0144,0.0035,AMZN
19031,2021-12-30 00:00:00,3394.000000,3417.760010,3370.479980,3372.889893,1879200,0.0,0.0,0.0053,0.0037,AMZN


In [345]:
# Financial Data
# Wantede Stocks Contain Some IPOs
company_dailys.groupby("company").count().sort_values("Open", ascending=False)

,pk,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Volatility,Turnover
company,,,,,,,,,,,
VOX,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777
VDC,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777
VCR,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777,3777
JD,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518
VDE,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518,2518
...,...,...,...,...,...,...,...,...,...,...,...
RSI,426,426,426,426,426,426,426,426,426,426,426
ADTX,381,381,381,381,381,381,381,381,381,381,381
RKT,355,355,355,355,355,355,355,355,355,355,355


In [346]:
# Comments
with sql.connect("../data/interim/discord/discord.db") as con:
    emote_df = pd.read_sql("SELECT * FROM comments WHERE emojis != ''", con=con)

In [351]:
# Comments
emote_agg = {}
def agg_emojis(comma_sep_emotes):
    for e in comma_sep_emotes.split(","):
        if e in emote_agg.keys():
            emote_agg[e] += 1
        else:
            emote_agg[e] = 1

In [352]:
# Comments
emote_df.emojis.apply(agg_emojis)

0       None
1       None
2       None
3       None
4       None
        ... 
2331    None
2332    None
2333    None
2334    None
2335    None
Name: emojis, Length: 2336, dtype: object

In [373]:
# Comments
emote_agg_df = pd.Series(emote_agg).reset_index(name="count").rename(columns={"index": "emote"})

In [375]:
# Comments
emoji_parse = emoji.UNICODE_EMOJI

In [383]:
# Comments
emote_agg_df = emote_agg_df.assign(unicode_name = lambda e: e.emote.apply(lambda x: emoji_parse[x])).sort_values("count", ascending=False)

In [388]:
# with sql.connect("../data/interim/discord/discord.db") as con:
#     emote_agg_df.to_sql("chatEmotes", con=con, index_label='pk', if_exists="replace")

In [106]:
# Comments
tickers_array = company_dailys.symbol.unique()

In [107]:
# Financial Data
def stringNone(na):
    return na if na != None else ""

In [113]:
# Fiancial Data
# Yahoo API
keys = ["quoteType", "longName", "shortName", "logo_url", "sector", "industry", "holdings", "fundFamily", "category"]
k = 77
# company_info_df = pd.DataFrame()
for t in tickers_array[77:]:
    try:
        tick_info = yf.ticker.Ticker(t).get_info()
    except:
        print("Sleeping...")
        time.sleep(90)
        tick_info = yf.ticker.Ticker(t).get_info()
    
    for key in keys:
        company_info_df.loc[k, key] = stringNone(tick_info.get(key))
    k+= 1
    print(f"{t} Done.")

VOX Done.
VCR Done.
VDC Done.
VDE Done.
VFH Done.
VHT Done.
VIS Done.
VGT Done.
VAW Done.
VNQ Done.
VPU Done.


In [116]:
# Financial Data
# Yahoo API info
company_info_df.loc[:, "sector"] = company_info_df.sector.apply(lambda v: v if v != "" else np.nan).fillna(company_info_df.category)

In [117]:
# Financial Data
# Yahoo API info
company_info_df.loc[:, "symbol"] = pd.Series(tickers_array)

In [118]:
# Financial Data
# Sector Names
company_info_df[lambda x: (x.quoteType == "EQUITY")].sector.unique()

array(['Consumer Cyclical', 'Technology', 'Financial Services',
       'Healthcare', 'Communication Services', 'Industrials',
       'Consumer Defensive', 'Basic Materials', 'Utilities'], dtype=object)

In [119]:
# Fiancial Data
# Info
company_info_df.iloc[-15:, :]

,quoteType,longName,shortName,logo_url,sector,industry,holdings,fundFamily,category,symbol
73,ETF,Invesco QQQ Trust,"Invesco QQQ Trust, Series 1",,Large Growth,,"[{'symbol': 'AAPL', 'holdingName': 'Apple Inc'...",Invesco,Large Growth,QQQ
74,ETF,SPDR S&P 500 ETF Trust,SPDR S&P 500,,Large Blend,,"[{'symbol': 'AAPL', 'holdingName': 'Apple Inc'...",SPDR State Street Global Advisors,Large Blend,SPY
75,ETF,iPath Series B S&P 500 VIX Short-Term Futures ETN,iPath Series B S&P 500 VIX Shor,,Trading--Miscellaneous,,[],Milleis Investissements Funds,Trading--Miscellaneous,VXX
76,ETF,ProShares UltraPro Short QQQ,ProShares UltraPro Short QQQ,,Trading--Inverse Equity,,"[{'symbol': '', 'holdingName': 'Nasdaq 100 Ind...",ProShares,Trading--Inverse Equity,SQQQ
77,ETF,Vanguard Communication Services Index Fund ETF...,Vanguard Communication Services,,Communications,,"[{'symbol': 'FB', 'holdingName': 'Facebook Inc...",Vanguard,Communications,VOX
78,ETF,Vanguard Consumer Discretionary Index Fund ETF...,Vanguard Consumer Discretion ET,,Consumer Cyclical,,"[{'symbol': 'AMZN', 'holdingName': 'Amazon.com...",Vanguard,Consumer Cyclical,VCR
79,ETF,Vanguard Consumer Staples Index Fund ETF Shares,Vanguard Consumer Staples ETF,,Consumer Defensive,,"[{'symbol': 'PG', 'holdingName': 'Procter & Ga...",Vanguard,Consumer Defensive,VDC
80,ETF,Vanguard Energy Index Fund ETF Shares,Vanguard Energy ETF,,Equity Energy,,"[{'symbol': 'XOM', 'holdingName': 'Exxon Mobil...",Vanguard,Equity Energy,VDE
81,ETF,Vanguard Financials Index Fund ETF Shares,Vanguard Financials ETF,,Financial,,"[{'symbol': 'JPM', 'holdingName': 'JPMorgan Ch...",Vanguard,Financial,VFH
82,ETF,Vanguard Health Care Index Fund ETF Shares,Vanguard Health Care ETF,,Health,,"[{'symbol': 'JNJ', 'holdingName': 'Johnson & J...",Vanguard,Health,VHT


In [120]:
# Financial Data
# map yahoo sector to Like GICS sector
sectors = {
    "Large Growth": "Technology",
    "Large Blend": "Mutual Fund",
    "Trading--Miscellaneous": "Inverse Equity",
    "Trading--Inverse Equity": "Inverse Equity",
    "Communications": "Communications Services",
    "Equity Energy": "Energy",
    "Financial": "Financials",
    "Financial Services": "Financials",
    "Health": "Healthcare",
    'Basic Materials': 'Materials',
    "Natural Resources": "Materials",
    "Consumer Defensive": "Consumer Staples",
    "Consumer Cyclical": "Consumer Discretionary",
}

In [121]:
# Financial Data
# 
company_info_df_w_sectors = company_info_df.copy(deep=True)
company_info_df_w_sectors.loc[:, "sector"] = company_info_df_w_sectors.loc[:, "sector"].apply(lambda x: x if x not in sectors.keys() else sectors[x])

In [122]:
# Financial Data
# Unique Asset. Maybe Currency? 
company_info_df_w_sectors[lambda f: f.sector == ""]

,quoteType,longName,shortName,logo_url,sector,industry,holdings,fundFamily,category,symbol
72,CRYPTOCURRENCY,,Bitcoin USD,,,,,,,BTC-USD


In [123]:
# Financial Data
# Info
company_info_df_w_sectors.loc[72, "logo_url"] = "https://bitcoin.org/img/icons/opengraph.png?1641218872"

In [127]:
# News Articles
cleaned_articles = pd.read_csv("../data/interim/articlesCK.csv")

In [149]:
# Financial Data
# Info
equitys = company_info_df_w_sectors[lambda x: x.quoteType == "EQUITY"].symbol.values

In [174]:
# Analyst Recommendations
recommendation_history = pd.DataFrame()
for sym in equitys[29:]:
    try:
        recom = yf.ticker.Ticker(sym).get_recommendations()
    except:
        time.sleep(60)
        recom = yf.ticker.Ticker(sym).get_recommendations()
    
    if type(recom) != pd.DataFrame:
        next
    else:
        recom = recom.reset_index().assign(symbol=sym)
        recommendation_history = pd.concat([recommendation_history, recom], axis=0, ignore_index=True)
    
    print("Done with {}".format(sym))

Done with WWR
Done with HYLN
Done with SNDL
Done with JD
Done with SNAP
Done with RKT
Done with ES
Done with JKS
Done with NNDM
Done with GME
Done with SE
Done with IT
Done with MA
Done with KO
Done with MARA
Done with HEAR
Done with CRM
Done with CGC
Done with IZEA
Done with CBAT
Done with INTC
Done with SPCE
Done with AAL
Done with SPI
Done with ON
Done with AREC
Done with NFLX
Done with UONE
Done with BLNK
Done with HD
Done with ING
Done with DIS
Done with MO
Done with TLRY
Done with ADTX
Done with LMNL
Done with JMIA
Done with EXAS
Done with SOLO
Done with XERS
Done with DPW
Done with GILD
Done with VLDR


In [195]:
# Analyst Recommendations
pd.concat([recommendation_history["To Grade"], recommendation_history["From Grade"]], axis=0).unique()

array(['Overweight', 'Hold', 'Buy', 'Neutral', 'Underperform',
       'Outperform', 'Equal-Weight', 'Sell', '', 'Underweight',
       'Sector Perform', 'Market Perform', 'Perform', 'Sector Weight',
       'In-Line', 'Underperformer', 'Market Outperform', 'Peer Perform',
       'Negative', 'Positive', 'Sector Outperform', 'Strong Buy',
       'Long-term Buy', 'Long-Term Buy', 'Equal-weight', 'Reduce',
       'Accumulate', 'Mixed', 'Fair Value', 'Trim', 'Outperformer',
       'Top Pick', 'Speculative Buy', 'Average', 'Market Underperform',
       'Market Weight', 'Below Average', 'Hold Neutral', 'Conviction Buy',
       'Sector Underperform', 'Sector Performer', 'Above Average'],
      dtype=object)

In [211]:
# Analyst Recommendations
# Translate Differing Analyst terms into an interval scale [1, 5]
analyst_dict = {
    'Overweight': 'Bullish',
    'Hold': 'Neutral',
    'Buy': 'Very Bullish',
    'Neutral': 'Neutral',
    'Underperform': 'Bearish',
    'Outperform': 'Bullish',
    'Equal-Weight': 'Neutral',
    'Sell': 'Very Bearish',
    '': '',
    'Underweight': 'Bearish',
    'Sector Perform': 'Neutral',
    'Market Perform': 'Neutral',
    'Perform': 'Neutral', 'Sector Weight': 'Neutral',
    'In-Line': 'Neutral', 'Underperformer': 'Bearish',
    'Market Outperform': 'Bullish',
    'Peer Perform': 'Neutral',
    'Negative': 'Bearish',
    'Positive': 'Bullish',
    'Sector Outperform': 'Bullish',
    'Strong Buy': 'Very Bullish',
    'Long-term Buy': 'Very Bullish',
    'Long-Term Buy': 'Very Bullish',
    'Equal-weight': 'Neutral',
    'Reduce': 'Bearish',
    'Accumulate': 'Bullish',
    'Mixed': 'Neutral',
    'Fair Value': 'Neutral',
    'Trim': 'Bearish',
    'Outperformer': 'Bullish',
    'Top Pick': 'Very Bullish',
    'Speculative Buy': "Bullish",
    'Average': 'Neutral',
    'Market Underperform': 'Bearish',
    'Market Weight': 'Neutral',
    'Below Average': 'Bearish',
    'Hold Neutral': 'Neutral',
    'Conviction Buy': 'Very Bullish',
    'Sector Underperform': 'Bearish',
    'Sector Performer': 'Neutral',
    'Above Average': 'Bullish'
}

In [199]:
# Analyst Recommendations
recommendation_history.shape

(9388, 6)

In [210]:
# Analyst Recommendations
recommendation_history = recommendation_history[lambda x: x["To Grade"] != ""]

In [215]:
# Analyst Recommendations
recommendation_history = recommendation_history.assign(new_grade = lambda x: x["To Grade"].apply(lambda c: analyst_dict[c])).assign(prev_grade = lambda x: x["From Grade"].apply(lambda c: analyst_dict[c]))

In [217]:
# Analyst Recommendations
# Action types = upgrade, downgrade, or no change
recommendation_history.Action.unique()

array(['main', 'down', 'init', 'up', 'reit'], dtype=object)

In [242]:
# Analyst Recommendations
yt = recommendation_history.copy(deep=True)

In [250]:
# Analyst Recommendations
up_and_down = {'up': {'Very Bullish': 'Bullish', 'Bullish': 'Neutral', 'Neutral': 'Bearish', 'Bearish': 'Very Bearish'},
'down': {'Bullish': 'Very Bullish', 'Neutral': 'Bullish', 'Bearish': 'Neutral', 'Very Bearish': 'Bearish'}}

In [251]:
# Analyst Recommendations
for i, r in recommendation_history.iterrows():
    grade = r["prev_grade"]
    act = r["Action"]
    if grade == '':
        if act in ['main', 'reit', 'init']:
            recommendation_history.loc[i, 'prev_grade'] = r['new_grade']
        elif act == 'down':
            recommendation_history.loc[i, 'prev_grade'] = up_and_down['down'][r['new_grade']]
        elif act == 'up':
            recommendation_history.loc[i, 'prev_grade'] = up_and_down['up'][r['new_grade']]

In [ ]:
# recommendation_history, company_info_df_w_sectors, daily table from stocks and articlesCK

In [260]:
with sql.connect('../data/interim/stocks.db') as con:
    dailys_df = pd.read_sql("SELECT * FROM daily", con=con)

In [264]:
# Financial Data
dailys_df = dailys_df.drop('pk', axis=1)

In [266]:
# News Articles
cleaned_articles = cleaned_articles.drop('pk', axis=1)

In [271]:
# Comments
mentions = pd.read_csv('../data/interim/trading_counts.csv').rename(columns={'company': 'symbol'})

In [278]:
# News Articles
cleaned_articles = cleaned_articles.rename(columns={'ticker': 'symbol'})

In [281]:
# Financial Data
dailys_df = dailys_df.rename(columns={'company': 'symbol'})

In [ ]:
# Financial Data
dailys_df.Date = pd.to_datetime(dailys_df.Date)

In [290]:
# News Articles
cleaned_articles.date = pd.to_datetime(cleaned_articles.date)

In [296]:
# Save to SQL
with sql.connect('../data/raw/companies.db') as con:
    dailys_df.to_sql("daily", con=con, index=True, index_label='pk', if_exists='replace')
    recommendation_history.to_sql('recommendations',  con=con, index=True, index_label='pk', if_exists='replace')
    company_info_df_w_sectors.drop('holdings', axis=1).to_sql('info', con=con, index=True, index_label='pk', if_exists='replace')
    cleaned_articles.to_sql('info', con=con, index=True, index_label='pk', if_exists='replace')
    mentions.to_sql('mentions', con=con, index=True, index_label='pk', if_exists='replace')

In [11]:
# News Articles scrape for Seeking Alpha
driver = webdriver.Chrome("../../../Python/scraping/chromedriver2.exe", desired_capabilities=caps)
driver.get("https://seekingalpha.com/symbol/HEAR/news")
time.sleep(4.2)
button = driver.find_element(By.CSS_SELECTOR, 'button[data-test-id="header-button-sign-in"]')
button.click()
button_g = driver.find_element(By.CSS_SELECTOR, 'button[data-test-id="sign-in-with-google"]')
time.sleep(4.2)
button_g.click()
email = driver.find_element(By.CSS_SELECTOR, '[type="email"]')
time.sleep(22.3)
email.send_keys("kylel9815@gmail.com")
time.sleep(10)
driver.quit()

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [163]:
# News Articles
with open('sqqq.htm', encoding='utf-8') as fp:
    soup = bs4.BeautifulSoup(fp.read(), 'html5lib')
    for city in soup.find_all('h3'):
        print(city.text)

Nasdaq 100 slides at the open, which means three inverse QQQ funds are rallying
BofA fund manager survey triggers first sell signal since just before February crash
Earnings season was a smasher amid election 2020 distraction
QQQs head lower after hours as Apple and Amazon slip post-earnings


In [98]:
# News Articles
# Scraping attempt
equities = company_info_df_w_sectors.symbol.apply(str.lower).values
copied = "?from=2020-05-01T16%3A00%3A00.000Z&to=2021-09-01T16%3A00%3A00.000Z"
for e in equities:
    letters = e.split('')
    pyautogui.moveTo(2499, 135, duration=2)
    pyautogui.click()
    pyautogui.typewrite(letters + ['enter'], .5)
    pyautogui.moveTo(2406, 433, duration=2)
    pyautogui.click()
    pyautogui.moveTo(2424, 47, duration=1.2)
    pyautogui.click(clicks=3, interval=1)
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.hotkey('enter')
    time.sleep(5)
    for l in range(1000):
        pyautogui.keyDown('pagedown')
        if l % 200 == 0:
            print(f'{l} at {dt.datetime.fromtimestamp(time.time()).strftime("%I:%M:%S")}')
    pyautogui.moveTo(2079, 590, duration=1.2)
    pyautogui.hotkey('ctrl', 's')
    time.sleep(2)
    pyautogui.typewrite([letters]+['.', 'm', 'h', 't', 'm', 'l', 'enter'], .15)
    time.sleep(2)
    pyautogui.hotkey('enter')
    time.sleep(3)
    pyautogui.hotkey('home')
    time.sleep(2)

0 at 02:43:15
100 at 02:43:26
200 at 02:43:37
300 at 02:43:48


In [59]:
# News Articles
# Saved Text Files
for folder, sf, files in os.walk(r"H:\Big Data\articles"):
    for f in files[:5]:
        if f.endswith(".htm"):
            f_name = "H:\\Big Data\\articles\\"  + f
            with open(f_name, encoding='latin-1') as file:
                soup = bs4.BeautifulSoup(file.read(), 'html5lib')
                print("="*18)
                print(f.replace(".htm", "").upper())
                print("="*18)
                for headline in soup.find_all('div', {'div':'[data-test-id="content-container"]'}):
                    print(headline.text.replace("\n", " "))

AAL
AAPL
ACB
ADTX
AMD


In [291]:
# News Articles
len(articles)

772

In [289]:
# News Articles Setup
    # soup.select('div[data-test-id="content-container"]') # article content
    # soup.select('h3[data-test-id="post-list-item-title"]') # title
    # soup.select('span[data-test-id="post-list-comments"]') # engagement
    # soup.select('span[data-test-id="post-list-date"]') # date
    # soup.select('span[data-test-id="post-list-author"]') # author

In [343]:
# News Articles
# Scrape Files
crypt_articles = pd.DataFrame()
columns = ['content', 'headline', 'link', 'comments', 'date',  'symbol', 'publisher']
art_links = []
o = 0
for folder, sf, files in os.walk(r"H:\Big Data\articles"):
    for file in files:
        if file.endswith(".htm"):
            art_links.append(file)

for sym in art_links:
    soup = bs4.BeautifulSoup(open(f"H:\\Big Data\\articles\\{sym}", encoding='latin-1').read().replace("\n", ""), 'html5lib')
    sym = sym.replace(".htm", "")
    articles = soup.select('article[data-test-id="post-list-item"]') 
    for a in articles:
        cont =  a.select('div[data-test-id="content-container"]') # article content
        headline = a.select('h3[data-test-id="post-list-item-title"]') # title
        comms = a.select('a[data-test-id="post-list-comments"]') # engagement
        date_art = a.select('span[data-test-id="post-list-date"]') # date
        link =  a.select('h3[data-test-id="post-list-item-title"]')[0].select('a')[0].attrs['href'] # article link
        symbol = sym
        publisher = 'SA'
        scrapes = [cont, headline, comms, date_art]
        scraped = []
        for s in scrapes:
            if len(s) != 0:
                scraped.append(s[0].text)
            else:
                scraped.append('')
        
        
        crypt_articles.loc[o, ['article', 'headline', 'comments', 'date', 'link', 'symbol', 'publisher']] = scraped + [link, sym.upper(), 'Seeking Alpha']
        o+=1
        

In [301]:
# News Articles
with sql.connect('../data/interim/companies.db') as con:
    arts = pd.read_sql("select * from articles", index_col='pk', con=con)

In [7]:
# News Articles
crypt_articles = crypt_articles.assign(comments = lambda x: x.comments.apply(str)).assign(comments = lambda x: x.comments.apply(lambda c: re.sub(r"Comment(s)?", "", c)))

In [8]:
# News Articles
crypt_articles = crypt_articles.assign(headline = lambda x: x.headline.apply(str.strip)).assign(article = lambda x: x.article.apply(str.strip))

In [9]:
# News Articles
crypt_articles.loc[:, "date"] = pd.to_datetime(crypt_articles.date)

In [20]:
# News Articles
# Encrypt to Conform to Subscription TOS
crypt_articles = crypt_articles.assign(article = lambda x: x.article.apply(bytes, encoding='utf-8')).assign(article = lambda x: x.article.apply(cryptor.encrypt))
crypt_articles = crypt_articles.assign(headline = lambda x: x.headline.apply(bytes, encoding='utf-8')).assign(headline = lambda x: x.headline.apply(cryptor.encrypt))
crypt_articles = crypt_articles.assign(link = lambda x: x.link.apply(bytes, encoding='utf-8')).assign(link = lambda x: x.link.apply(cryptor.encrypt))
crypt_articles = crypt_articles.assign(comments = lambda x: x.comments.apply(bytes, encoding='utf-8')).assign(comments = lambda x: x.comments.apply(cryptor.encrypt))
crypt_articles = crypt_articles.assign(date = lambda x: x.date.apply(str).apply(bytes, encoding='utf-8')).assign(date = lambda x: x.date.apply(cryptor.encrypt))
crypt_articles = crypt_articles.assign(publisher = lambda x: x.publisher.apply(str).apply(bytes, encoding='utf-8')).assign(publisher = lambda x: x.publisher.apply(cryptor.encrypt))
crypt_articles = crypt_articles.assign(symbol = lambda x: x.symbol.apply(str).apply(bytes, encoding='utf-8')).assign(symbol = lambda x: x.symbol.apply(cryptor.encrypt))

In [57]:
# News Articles
with sql.connect('../data/raw/crypt.db') as con:
    crypt_articles.to_sql("crypt_articles", index=True,index_label='pk', con=con, if_exists='replace')

In [5]:
# News Articles
with sql.connect('../data/interim/companies.db') as con:
    articles = pd.read_sql("select * from articles", index_col="pk", con=con)

In [10]:
# News Articles
articles

,id,title,link,date,publisher,symbol,article
pk,,,,,,,
0,2467735,Tesla tells China car cameras not activated ou...,https://www.investing.com/news/economy/tesla-s...,2021-04-07 00:00:00,Reuters,TSLA,Cameras in Tesla (NASDAQ:TSLA) cars are not ac...
1,2469136,Tesla lashes out at German red tape ahead of p...,https://www.investing.com/news/stock-market-ne...,2021-04-08 00:00:00,Reuters,TSLA,Tesla (NASDAQ:TSLA) sharply criticised lengthy...
2,2467806,"Sun, sea and cybernauts: the long road for Gre...",https://www.investing.com/news/technology-news...,2021-04-07 00:00:00,Reuters,TSLA,"From her garden on a sunny Greek island, Rowen..."
3,2467802,CEO Mary Barra bets GM can grow beyond cars an...,https://www.investing.com/news/stock-market-ne...,2021-04-07 00:00:00,Reuters,TSLA,Pam Fletcher wants to change the way General M...
4,2467233,"S&P 500 Flirts With Record, But 'Overbought' J...",https://www.investing.com/news/stock-market-ne...,2021-04-06 00:00:00,Investing.com,TSLA,The S&P 500 continued to trade near record hig...
...,...,...,...,...,...,...,...
11479,1759417,Stocks lower ahead of action-packed week,https://www.investing.com/news/stock-market-ne...,2019-01-28 00:00:00,Seeking Alpha,SQQQ,Global shares are starting the week on the bac...
11480,1748359,"Stocks weather Brexit debacle, pound steadies",https://www.investing.com/news/stock-market-ne...,2019-01-16 00:00:00,Seeking Alpha,SQQQ,Markets and the pound appear to have weathered...
11481,1746970,Equities boosted by China stimulus announcement,https://www.investing.com/news/stock-market-ne...,2019-01-15 00:00:00,Seeking Alpha,SQQQ,Stocks gained ground overnight as China announ...


In [114]:
# Sentiment Analysis
sia = SentimentIntensityAnalyzer()

In [115]:
# Sentiment Analysis
def sentiment_art(art):
    return sia.polarity_scores(art.replace("  ", '').replace('\x92', "'"))

In [21]:
# News Articles
articles = articles.assign(compound_sent = lambda x: x.article.apply(sentiment_art))

In [32]:
# News Articles
articles= articles.assign(pos_sent = lambda x: x.compound_sent.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.compound_sent.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.compound_sent.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.compound_sent.apply(lambda y: y['compound'])).drop("compound_sent", axis=1, inplace=True)

In [85]:
# News Articles
# Seeking Alpha
crypt_articles = crypt_articles.assign(sentiment = lambda x: x.article.apply(cryptor.decrypt).apply(str, encoding='utf-8').apply(sentiment_art))

In [86]:
# News Articles
crypt_articles = crypt_articles.assign(pos_sent = lambda x: x.sentiment.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.sentiment.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.sentiment.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.sentiment.apply(lambda y: y['compound']))

In [87]:
# News Articles
crypt_articles.drop("sentiment", axis=1, inplace=True)

In [88]:
# News Articles
crypt_articles

,article,headline,comments,date,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent
pk,,,,,,,,,,,
0,b'GPRLBtWdnY_RZcIhFw1I_zO6kgjSRPGbPiJtEEq0Vfs=...,b'Jqc6cDZhIcu6aNfkqyzfELn7KxIpIge0W1jApWaJ_iQ=...,b'YlLzFOG-Xec6-61v5jY0E2pD1Eo572iW8WYx99XeBfE=...,b'CuGllAvrxyYSr0-fB0EuU07l-QRZIQEdbewugAs0kgM=...,b'qaEtC-ufuGelB1bHBC1wVj_gVPxagArmLAVwkzeRc1A=...,b'2N7lFkxliIikw5YxhCE-xZZuoOBcsNzZ7w_8q8WJh24=...,b'iEqM7yp5q1dD9E5ffh-q3I8i3Ot9RN-9TW6mHdb7Hik=...,0.111,0.029,0.861,0.9325
1,b'wIN9CiTc_XLewNlpYPGLvPRbCWRV6d2eIoUPXS-j-Ko=...,b'MmssHZw5N6QiPfWPe_1sS53s4vLgtpnmul7xh63617I=...,b'C--kMS-JiqtjT9irkZM9ArUCXRICre3raTElsaqfKyk=...,b'Ol_501yn3fCC0QCcOjLUtnrO8Ix-BXy-z1BmDtI7Agk=...,b'CRZ4uDkFkfGBUd2un4QtzL4qbgOgdQmdHjzrg85uSLI=...,b'e4_N0A8W-YST2-PPh5bPDojLjDRVgx6LswFDTfg8mqQ=...,b'4vqaOCcT1K6yPtnKEj_35QcyK6PO8fdhgTtFFac61NQ=...,0.073,0.024,0.904,0.8360
2,b'PK1nnYlzL3TekPT49yIf75Ao9zjkAcZK-IfbvnBMLl8=...,b'dSI3TD_qWPddyrgY1lLKfq6GPKHo-nUIWsbmHAVCl8U=...,b'fNJoAgeOc0YqoVRXEX0Wt4DuTTb2UvojOG6nPDWBA0w=...,b'P-P5di0wLHphlX_MkXnaacCaJPVY4ggKxc7aX_hov3s=...,b'0U7mMfkmVgtoosoO_VZukRA0K7JWnaG42jEQnytwT_E=...,b'vUme-p7pQwYUp4k4lnwqD3HeJLWFql4XMGgWxt5NGXI=...,b'3iYHSCuqjIl1c_e9PG8yPpjtvxhIF5s9huWS5QE-WT4=...,0.070,0.051,0.879,0.5106
3,b'CPMuf42Elv6xCSyBMukTma8Qv6la83EgjcjNiXFTwxU=...,b'hG9-8gAJzpPFcJoriTRXfGtkd9iLHqUDJEHR9qBdY8o=...,b'zyelglAChrb2pGJBkxGXv4b79GaMYOTuCGLhzqrO3Jw=...,b'ijrnlZOKYXHQ-_YEN84BD66TIXA9qK9YOhZ2rzVM33g=...,b'dvD9UiWPefqPqmLBkaHcnKVI2AkU0yat-MWCXBbV5HY=...,b'amZas7P4RguV7fhchUiInz6ptmIU9fUgfI7ffyxSyJc=...,b'SvGwk9y1zXolFtEeRsql0T3o-6QUl-KPDZRFzJvaBjw=...,0.050,0.032,0.917,0.6365
4,b'APDsLi1C-5Cs8qMKED1HGGP3_ELt-243bE90jNgCSt8=...,b'uXqeB7Z26UVUMe9lWDn7aJNKt85DJYCGy76zpglG0MQ=...,b'crbCNwpk8otn8PhgtUYtbkj0ZoYJskf-6h79kAgM3cU=...,b'rs0e1220jKHb13-yvX3IROuRhkr39mZbcBCzximkXpc=...,b'G9w5fHXsM5YcrNzMme-ZeMvZGc-YsIIR9WarI8OaeQQ=...,b'yHd4h4VIxdMJvtAys5KVCRTy3fCPF4nWlJ6HiYdzeCU=...,b'Y4DJ1EhgR7YfxiSxQ-g2_70CVO7r4AXjt-MQQOWgraU=...,0.075,0.049,0.876,0.6577
...,...,...,...,...,...,...,...,...,...,...,...
15321,b'ekFkZh6D0fKhtrX9JWTyMWqsxk5Tvkbr9DLlhKefioA=...,b'lT4Z1qmrzbC5YrbugaSrI3sG_Q0MZjTAiUiNoJjGLvU=...,b'waI0mWynnA6486ldfqBhoWyvcrE4SsU0IZOc6Pj_59E=...,b'TOrrZfIXqoSTCDm93udYnQwaP2G9TmhgJQFG0oluX-E=...,b'BzPcf6DTOQI22CxuleVqYkWhXlc1JdrAS0hKzrQctmc=...,b'3DEEiE1Ce3Z2quhmRpHPQBKJDkVmpofZIhjW-RrFnx4=...,b'9HsUPZeESNABVDzysHjjfAtHrYQO6yvDizzAfJLW9JA=...,0.160,0.028,0.812,0.9307
15322,b'rLXMpTb1i59gaFOEkkgvAD3kT5aaOuxnF6fcjsQLLeM=...,b'4FMTpvLJr0R3WpM2f6c67OBAE0xcIECRGMEy1Q3uwi0=...,b'BU2FvsM3asD_-1E6yRCWn-quHT70p5_szt5IYbpnQJM=...,b'rp5SfaPJ0UrnTislTtU2IcLfGZJ9kRVP76zElyqsNzM=...,b'TcPy2tRPgK_q0bXMf8XlIRfaaFA2lsRL_kL3tmjrhnE=...,b'IbvPAn5Li7UTQzX730uKmlQ0Vgl27RUtdDsetgWXz1k=...,b'OrVjJUhI41jBz7EUlszp6A7kJNVv-XzR0OXffF8EERA=...,0.156,0.023,0.821,0.9118
15323,b'BUqL8lAJPC3bZXFcizOEGBBGWUab9SInPbMMPPEio4Y=...,b'WlLkwVXBuCumG2_1FCxpod6eMJTsuUXlInyQVMRZ6bU=...,b'lwK7TJ1IwgVjC_0yrmXQhoGFGlw1i15IvBziVCc52BE=...,b'GgfoxDEsvLBh5UVTv0Yas3spzazV2omdqO8dMLKC5fI=...,b'xPFqI9ISYLyvitmkvD_tn8F9zjnz5WegxBpKGonKH-s=...,b'6jHw_5PSh4HSubF4cqPEVKwgybGuwp92G57lMQlQlEk=...,b'zm1hG9ivgNhIKXYRZoYj2rERK7MYxbak8EzYEJnJ6hM=...,0.129,0.000,0.871,0.8834


In [97]:
# News Articles
crypt_articles.loc[:, 'article'] = crypt_articles.article.apply(cryptor.decrypt).apply(str, encoding='utf-8').apply(lambda x: x.replace('  ', '').replace('\x92', "'")).apply(bytes, encoding='utf-8').apply(cryptor.encrypt)

In [98]:
# News Articles
crypt_articles

,article,headline,comments,date,link,symbol,publisher,pos_sent,neg_sent,neu_sent,comp_sent
pk,,,,,,,,,,,
0,b'irJmPVY100-TVuNIsOyq99aC9s9oFX4l1Snzs47oW60=...,b'Jqc6cDZhIcu6aNfkqyzfELn7KxIpIge0W1jApWaJ_iQ=...,b'YlLzFOG-Xec6-61v5jY0E2pD1Eo572iW8WYx99XeBfE=...,b'CuGllAvrxyYSr0-fB0EuU07l-QRZIQEdbewugAs0kgM=...,b'qaEtC-ufuGelB1bHBC1wVj_gVPxagArmLAVwkzeRc1A=...,b'2N7lFkxliIikw5YxhCE-xZZuoOBcsNzZ7w_8q8WJh24=...,b'iEqM7yp5q1dD9E5ffh-q3I8i3Ot9RN-9TW6mHdb7Hik=...,0.111,0.029,0.861,0.9325
1,b'dm4lZQAFo6mpCER-TQjfKvTtxI7FC1fuvtnETRlAnhw=...,b'MmssHZw5N6QiPfWPe_1sS53s4vLgtpnmul7xh63617I=...,b'C--kMS-JiqtjT9irkZM9ArUCXRICre3raTElsaqfKyk=...,b'Ol_501yn3fCC0QCcOjLUtnrO8Ix-BXy-z1BmDtI7Agk=...,b'CRZ4uDkFkfGBUd2un4QtzL4qbgOgdQmdHjzrg85uSLI=...,b'e4_N0A8W-YST2-PPh5bPDojLjDRVgx6LswFDTfg8mqQ=...,b'4vqaOCcT1K6yPtnKEj_35QcyK6PO8fdhgTtFFac61NQ=...,0.073,0.024,0.904,0.8360
2,b'-WmOUAHApUlh3GC982zwTG8TJChUnK4BZf0y7yR35ds=...,b'dSI3TD_qWPddyrgY1lLKfq6GPKHo-nUIWsbmHAVCl8U=...,b'fNJoAgeOc0YqoVRXEX0Wt4DuTTb2UvojOG6nPDWBA0w=...,b'P-P5di0wLHphlX_MkXnaacCaJPVY4ggKxc7aX_hov3s=...,b'0U7mMfkmVgtoosoO_VZukRA0K7JWnaG42jEQnytwT_E=...,b'vUme-p7pQwYUp4k4lnwqD3HeJLWFql4XMGgWxt5NGXI=...,b'3iYHSCuqjIl1c_e9PG8yPpjtvxhIF5s9huWS5QE-WT4=...,0.070,0.051,0.879,0.5106
3,b'9LSNvfqF7Q32Mpbdtl5FnxunGhycKQQ7-KsgipJKoTw=...,b'hG9-8gAJzpPFcJoriTRXfGtkd9iLHqUDJEHR9qBdY8o=...,b'zyelglAChrb2pGJBkxGXv4b79GaMYOTuCGLhzqrO3Jw=...,b'ijrnlZOKYXHQ-_YEN84BD66TIXA9qK9YOhZ2rzVM33g=...,b'dvD9UiWPefqPqmLBkaHcnKVI2AkU0yat-MWCXBbV5HY=...,b'amZas7P4RguV7fhchUiInz6ptmIU9fUgfI7ffyxSyJc=...,b'SvGwk9y1zXolFtEeRsql0T3o-6QUl-KPDZRFzJvaBjw=...,0.050,0.032,0.917,0.6365
4,b'tZhuCVTVSuIkM2LEoxRM0zbCfwCrFJbyu7aNbX7spjM=...,b'uXqeB7Z26UVUMe9lWDn7aJNKt85DJYCGy76zpglG0MQ=...,b'crbCNwpk8otn8PhgtUYtbkj0ZoYJskf-6h79kAgM3cU=...,b'rs0e1220jKHb13-yvX3IROuRhkr39mZbcBCzximkXpc=...,b'G9w5fHXsM5YcrNzMme-ZeMvZGc-YsIIR9WarI8OaeQQ=...,b'yHd4h4VIxdMJvtAys5KVCRTy3fCPF4nWlJ6HiYdzeCU=...,b'Y4DJ1EhgR7YfxiSxQ-g2_70CVO7r4AXjt-MQQOWgraU=...,0.075,0.049,0.876,0.6577
...,...,...,...,...,...,...,...,...,...,...,...
15321,b'L3Yd-S0WDzU1bHOnrlNMO5-vfp1mBniXdZKgwMbZlrs=...,b'lT4Z1qmrzbC5YrbugaSrI3sG_Q0MZjTAiUiNoJjGLvU=...,b'waI0mWynnA6486ldfqBhoWyvcrE4SsU0IZOc6Pj_59E=...,b'TOrrZfIXqoSTCDm93udYnQwaP2G9TmhgJQFG0oluX-E=...,b'BzPcf6DTOQI22CxuleVqYkWhXlc1JdrAS0hKzrQctmc=...,b'3DEEiE1Ce3Z2quhmRpHPQBKJDkVmpofZIhjW-RrFnx4=...,b'9HsUPZeESNABVDzysHjjfAtHrYQO6yvDizzAfJLW9JA=...,0.160,0.028,0.812,0.9307
15322,b'fau0eli751lInoJjPq9tJsdK6MVjBhnuLjKCfknx288=...,b'4FMTpvLJr0R3WpM2f6c67OBAE0xcIECRGMEy1Q3uwi0=...,b'BU2FvsM3asD_-1E6yRCWn-quHT70p5_szt5IYbpnQJM=...,b'rp5SfaPJ0UrnTislTtU2IcLfGZJ9kRVP76zElyqsNzM=...,b'TcPy2tRPgK_q0bXMf8XlIRfaaFA2lsRL_kL3tmjrhnE=...,b'IbvPAn5Li7UTQzX730uKmlQ0Vgl27RUtdDsetgWXz1k=...,b'OrVjJUhI41jBz7EUlszp6A7kJNVv-XzR0OXffF8EERA=...,0.156,0.023,0.821,0.9118
15323,b'ZXkbDgPSamX8tXBifISZRHlTt4zOw6eD3evGj8R6GGQ=...,b'WlLkwVXBuCumG2_1FCxpod6eMJTsuUXlInyQVMRZ6bU=...,b'lwK7TJ1IwgVjC_0yrmXQhoGFGlw1i15IvBziVCc52BE=...,b'GgfoxDEsvLBh5UVTv0Yas3spzazV2omdqO8dMLKC5fI=...,b'xPFqI9ISYLyvitmkvD_tn8F9zjnz5WegxBpKGonKH-s=...,b'6jHw_5PSh4HSubF4cqPEVKwgybGuwp92G57lMQlQlEk=...,b'zm1hG9ivgNhIKXYRZoYj2rERK7MYxbak8EzYEJnJ6hM=...,0.129,0.000,0.871,0.8834


In [16]:
# News Articles
# Combine Article Sources into One
with sql.connect("../data/interim/companies.db") as con:
    arts = pd.read_sql(f"SELECT * from articles", con=con, parse_dates={'date': '%Y-%m-%d %H:%M:%S'}).drop('id', axis=1)

with sql.connect("../data/raw/crypt.db") as con:
    second_arts = pd.read_sql('SELECT pk, headline as title, link, date, publisher, symbol, article, pos_sent, neg_sent, neu_sent, comp_sent FROM crypt_articles;', con=con)#.applymap(cryptor.decrypt)
    decrypt_cols = [x for x in second_arts.columns if x not in ['pk', 'pos_sent', "neu_sent", "neg_sent", "comp_sent"]]
    for col in decrypt_cols:
        second_arts.loc[:, col] = second_arts.loc[:, col].apply(bytes).apply(cryptor.decrypt).apply(str, encoding='utf-8')
        if col == 'date':
            second_arts.loc[:, col] = second_arts.loc[:, col].apply(str.split, sep=" ").apply(lambda x: x[0]).apply(pd.to_datetime)

arts = pd.concat([arts, second_arts], axis=0, ignore_index=True).sort_values('date')        

In [19]:
# News Articles
arts.groupby('symbol').agg(['first', 'last', 'count']).date.sort_values('first', ascending=False)

,first,last,count
symbol,,,
CRSR,2020-09-14,2021-08-31,39
RSI,2020-07-27,2021-08-30,41
VLDR,2020-06-27,2021-08-31,99
HYLN,2020-06-19,2021-08-31,65
PT,2020-06-15,2021-08-31,15
...,...,...,...
HEAR,2018-11-07,2021-11-29,61
SOLO,2018-10-29,2021-12-14,80
XERS,2018-10-23,2021-12-31,58


In [37]:
# Comments
with sql.connect("../data/interim/companies.db") as con:
    mentions = pd.read_sql(f"SELECT * from mentions", con=con, index_col='pk')

with sql.connect("../data/interim/discord/discord.db") as con:
    comments = pd.read_sql(f"SELECT * from comments", con=con, index_col='pk')

In [59]:
# Comments
mentioned_companies = mentions.symbol.values[:100]

In [72]:
# Comments
mentions.iloc[:100, :]

,symbol,counts
pk,,
0,SPY,167
1,TSLA,166
2,NIO,151
3,AMD,133
4,AAPL,114
...,...,...
95,SOLO,15
96,XERS,15
97,DPW,14


In [124]:
# Comments
company_regex = re.compile(r"|".join([x for x in mentioned_companies]))

In [148]:
# Comments
# Top 100 Companies
value_comments = comments[lambda x: x.content.str.contains(company_regex)]

In [149]:
# Comments
value_comments.loc[:, 'timestamp'] = pd.to_datetime(comments[lambda x: x.content.str.contains(company_regex)].timestamp)

C:\Users\Kyle\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [150]:
# Comments
value_comments = value_comments[lambda x: (x.isBot==0)].reset_index().assign(symbols=lambda x: x.content.apply(lambda s: re.findall(company_regex, s)))

In [151]:
# Comments
value_comments=value_comments.assign(symbols=lambda x: x.symbols.apply(str))

In [152]:
# Comments
value_comments = value_comments.assign(content_cleaned = lambda x: x.content.apply(lambda s: re.sub(re.compile(r"<.+>|@everyone"), "", s)))

In [153]:
# Comments
value_comments=value_comments.assign(compound_sent = lambda x: x.content_cleaned.apply(sentiment_art))

In [154]:
# Comments
value_comments=value_comments.assign(pos_sent = lambda x: x.compound_sent.apply(lambda y: y['pos']))\
    .assign(neg_sent = lambda x: x.compound_sent.apply(lambda y: y['neg']))\
        .assign(neu_sent = lambda x: x.compound_sent.apply(lambda y: y['neu']))\
            .assign(comp_sent = lambda x: x.compound_sent.apply(lambda y: y['compound']))

In [155]:
# Comments
value_comments=value_comments.drop(["compound_sent", "content_cleaned"], axis=1)

In [156]:
# Comments
with sql.connect("../data/interim/companies.db") as con:
    value_comments.to_sql('symbol_comments', con=con,index=False, if_exists='replace')

In [157]:
# Financial Data
with sql.connect("../data/interim/companies.db") as con:
    info = pd.read_sql(f"SELECT * from info", con=con, index_col='pk')

In [175]:
# Financial Data
info.loc[info['sector']=='Communication Services', 'sector'] = "Communications Services"

In [162]:
# Financial Data
# Comparison Baselines
comps = info[lambda v: ((v.quoteType=='ETF') & (v.shortName.str.contains('Vanguard')) | (v.quoteType=='ETF') & (v.shortName.str.contains('SPDR')))].loc[:, ['shortName', 'quoteType', 'sector', 'symbol']]

In [178]:
# Financial Data
info[lambda v: (v.quoteType=='EQUITY')].loc[:, ['shortName', 'quoteType', 'sector', 'symbol']].merge(comps, on='sector', how='left')

,shortName_x,quoteType_x,sector,symbol_x,shortName_y,quoteType_y,symbol_y
0,"Tesla, Inc.",EQUITY,Consumer Discretionary,TSLA,Vanguard Consumer Discretion ET,ETF,VCR
1,NIO Inc.,EQUITY,Consumer Discretionary,NIO,Vanguard Consumer Discretion ET,ETF,VCR
2,"Advanced Micro Devices, Inc.",EQUITY,Technology,AMD,Vanguard Information Tech ETF,ETF,VGT
3,Apple Inc.,EQUITY,Technology,AAPL,Vanguard Information Tech ETF,ETF,VGT
4,DraftKings Inc.,EQUITY,Consumer Discretionary,DKNG,Vanguard Consumer Discretion ET,ETF,VCR
...,...,...,...,...,...,...,...
67,Electrameccanica Vehicles Corp.,EQUITY,Consumer Discretionary,SOLO,Vanguard Consumer Discretion ET,ETF,VCR
68,"Xeris Biopharma Holdings, Inc.",EQUITY,Healthcare,XERS,Vanguard Health Care ETF,ETF,VHT
69,"Ault Global Holdings, Inc.",EQUITY,Financials,DPW,Vanguard Financials ETF,ETF,VFH
70,"Gilead Sciences, Inc.",EQUITY,Healthcare,GILD,Vanguard Health Care ETF,ETF,VHT


In [169]:
# Financial Data
comps

,shortName,quoteType,sector,symbol
pk,,,,
74,SPDR S&P 500,ETF,Mutual Fund,SPY
77,Vanguard Communication Services,ETF,Communications Services,VOX
78,Vanguard Consumer Discretion ET,ETF,Consumer Discretionary,VCR
79,Vanguard Consumer Staples ETF,ETF,Consumer Staples,VDC
80,Vanguard Energy ETF,ETF,Energy,VDE
81,Vanguard Financials ETF,ETF,Financials,VFH
82,Vanguard Health Care ETF,ETF,Healthcare,VHT
83,Vanguard Industrials ETF,ETF,Industrials,VIS
84,Vanguard Information Tech ETF,ETF,Technology,VGT


In [182]:
# Financial Data
with sql.connect("../data/interim/companies.db") as con:
    info.reset_index(drop=True).to_sql("info", con=con, index_label='pk', index=True, if_exists='replace')

In [205]:
# Financial Data
# pull equity from info table with associated GICS Sector VIA JOIN:
with sql.connect("../data/interim/companies.db") as con:
    reader = pd.read_sql("SELECT * FROM info JOIN (SELECT shortName compName, sector, symbol compSymbol FROM info WHERE compName LIKE '%Vanguard%' OR '%SPDR%') USING (sector) WHERE symbol='TSLA'", con=con, index_col='pk')

In [206]:
# Financial Data
reader

,quoteType,longName,shortName,logo_url,sector,industry,holdings,fundFamily,category,symbol,compName,compSymbol
pk,,,,,,,,,,,,
0,EQUITY,"Tesla, Inc.","Tesla, Inc.",https://logo.clearbit.com/tesla.com,Consumer Discretionary,Auto Manufacturers,None,,,TSLA,Vanguard Consumer Discretion ET,VCR


In [108]:
# Analyst Recommendations
recs

,Date,symbol,Firm,new_grade,prev_grade,Action
0,2012-02-16 07:42:00,TSLA,JP Morgan,Bullish,Bullish,main
1,2012-02-16 13:53:00,TSLA,Wunderlich,Neutral,Bearish,down
2,2012-02-17 06:17:00,TSLA,Oxen Group,Very Bullish,Very Bullish,init
3,2012-03-26 07:31:00,TSLA,Wunderlich,Very Bullish,Bullish,up
4,2012-05-22 05:57:00,TSLA,Maxim Group,Very Bullish,Very Bullish,init
...,...,...,...,...,...,...
9363,2021-05-11 00:00:00,VLDR,Needham,Very Bullish,Very Bullish,main
9364,2021-07-20 00:00:00,VLDR,Baird,Neutral,Bullish,down
9365,2021-08-06 00:00:00,VLDR,Needham,Very Bullish,Very Bullish,main
9366,2021-11-05 00:00:00,VLDR,Needham,Very Bullish,Very Bullish,main


In [109]:
articles

,date,symbol,publisher,pos_sent,neu_sent,neg_sent,comp_sent
0,2021-04-07 00:00:00,TSLA,Reuters,0.114,0.843,0.044,0.9181
1,2021-04-08 00:00:00,TSLA,Reuters,0.067,0.822,0.111,-0.8458
2,2021-04-07 00:00:00,TSLA,Reuters,0.121,0.841,0.038,0.9966
3,2021-04-07 00:00:00,TSLA,Reuters,0.090,0.878,0.033,0.9957
4,2021-04-06 00:00:00,TSLA,Investing.com,0.137,0.815,0.048,0.9936
...,...,...,...,...,...,...,...
26805,2020-05-07 00:00:00,ZM,Seeking Alpha,0.160,0.812,0.028,0.9307
26806,2020-05-07 00:00:00,ZM,Seeking Alpha,0.156,0.821,0.023,0.9118
26807,2020-05-06 00:00:00,ZM,Seeking Alpha,0.129,0.871,0.000,0.8834
26808,2020-05-04 00:00:00,ZM,Seeking Alpha,0.044,0.944,0.012,0.4310


In [ ]:
# News Articles
for c in crypt_arts.columns:
    if not (c.endswith('_sent')):
        crypt_arts.loc[:, c] = crypt_arts.loc[:, c].apply(cryptor.decrypt).apply(str, encoding='utf-8')

In [107]:
symbol_comms.shape

(3394, 4)

In [5]:
# recommendations, symbol comments, and articles
with sql.connect("../data/interim/companies.db") as con:
    recs = pd.read_sql("SELECT Date, symbol, Firm, new_grade, prev_grade, Action FROM recommendations", con=con, )
    articles = pd.read_sql("SELECT date, symbol, publisher,pos_sent, neu_sent, neg_sent, comp_sent FROM articles", con=con)
    symbol_comms = pd.read_sql("SELECT timestamp, symbols, channel, comp_sent FROM symbol_comments", con=con, parse_dates="timestamp")

In [6]:
# save seekingalpha raw articles
with sql.connect("../data/raw/crypt.db") as con:
    crypt_arts = pd.read_sql("SELECT date, symbol, publisher, pos_sent, neu_sent, neg_sent, comp_sent FROM crypt_articles", con=con)
crypt_arts.loc[:, "date"] = crypt_arts.date.apply(lambda x: x + "00")